In [1]:
import requests
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import time
import warnings

In [2]:
plt.style.use('seaborn')
warnings.filterwarnings('ignore')

C:\Users\ledne\AppData\Local\Temp\ipykernel_18984\4211250061.py:1: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [3]:
# pd.set_option('display.max_rows', None)

# Data processing

In [4]:
df = pd.read_csv("reached_data.csv")

In [5]:
df.head(5)

,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description
0,Machine Learning Engineer,Дата-сайентист,ДОМ.РФ,Москва,NaN,NaN,NaN,Решать амбициозные задачи из области NLP. Увер...,Принимать технические и архитектурные решения ...,Нет опыта,False,2023-06-07T11:25:50+0300,https://api.hh.ru/vacancies/81645171?host=hh.ru,Полная занятость,81645171,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>ЧЕМ НЕОБХОДИМО ЗАНИМАТЬСЯ:</strong>...
1,Куратор на курс Start machine learning,Специалист технической поддержки,Карпов Курсес,Санкт-Петербург,50000.0,NaN,RUR,Умеете грамотно выражать свои мысли в письменн...,Модерировать чаты со студентами (до 300 челове...,От 1 года до 3 лет,False,2023-06-13T18:16:41+0300,https://api.hh.ru/vacancies/81866480?host=hh.ru,Полная занятость,81866480,False,"[{'name': 'Грамотность'}, {'name': 'Грамотная ...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>Karpov.courses — молодой EdTech стартап в с...
2,Junior Data scientist/Machine learning,Аналитик,СБЕР,Москва,NaN,NaN,NaN,Обучение классических моделей машинного обучен...,Анализ и использование различных источников да...,От 1 года до 3 лет,False,2023-06-02T12:01:47+0300,https://api.hh.ru/vacancies/81456638?host=hh.ru,Полная занятость,81456638,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ищем Data scientist в Блок Сервисы. Это Бло...
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,250000.0,300000.0,RUR,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,True,2023-05-29T16:19:08+0300,https://api.hh.ru/vacancies/81253945?host=hh.ru,Частичная занятость,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Задачи:</strong></p> <ul> <li>Приду...
4,Стажер-разработчик (Machine Learning),"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Знание классических алгоритмов и структур данн...,Решение сложных задач для улучшения жизни людей.,Нет опыта,False,2023-05-26T12:10:38+0300,https://api.hh.ru/vacancies/81090082?host=hh.ru,Стажировка,81090082,NaN,[{'name': 'Machine Learning'}],[],"{'id': 'flexible', 'name': 'Гибкий график'}",<p>Машинное обучение и нейронные сети использу...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2336 entries, 0 to 2335
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    2336 non-null   object 
 1   role                    2336 non-null   object 
 2   employer.name           2336 non-null   object 
 3   area.name               2336 non-null   object 
 4   salary.from             489 non-null    float64
 5   salary.to               237 non-null    float64
 6   salary.currency         552 non-null    object 
 7   snippet.requirement     2326 non-null   object 
 8   snippet.responsibility  2314 non-null   object 
 9   experience.name         2336 non-null   object 
 10  accept_temporary        2336 non-null   bool   
 11  published_at            2336 non-null   object 
 12  url                     2336 non-null   object 
 13  employment.name         2336 non-null   object 
 14  id                      2336 non-null   

### Id column

In [7]:
# Let's see how many unique values are in a column.
#
# Cмотрим, сколько уникальных значений в столбце.  
df.id.nunique()

1839

In [8]:
# We have non-unique job id.
#
# У нас есть не уникальные id вакансий.  
df.id.value_counts()

81397120    5
81397148    5
81555584    5
81211273    4
81496350    4
           ..
81977786    1
80654593    1
81343003    1
81479973    1
81600904    1
Name: id, Length: 1839, dtype: int64

In [9]:
# An example of a duplicated vacancy.
#
# Пример задублированной вакансии.  
df[df.id == 81397120]

,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description
280,ML-разработчик в группу исследований машинного...,"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Писать код на C++ и Python. Хорошо знаете C++ ...,"Наша команда заботится о том, чтобы пользовате...",От 3 до 6 лет,False,2023-06-01T10:24:01+0300,https://api.hh.ru/vacancies/81397120?host=hh.ru,Полная занятость,81397120,NaN,"[{'name': 'Python'}, {'name': 'Алгоритмы'}, {'...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Наша миссия - сделать поисковую выдачу каче...
301,ML-разработчик в группу исследований машинного...,"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Писать код на C++ и Python. Хорошо знаете C++ ...,"Наша команда заботится о том, чтобы пользовате...",От 3 до 6 лет,False,2023-06-01T10:24:01+0300,https://api.hh.ru/vacancies/81397120?host=hh.ru,Полная занятость,81397120,NaN,"[{'name': 'Python'}, {'name': 'Алгоритмы'}, {'...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Наша миссия - сделать поисковую выдачу каче...
326,ML-разработчик в группу исследований машинного...,"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Писать код на C++ и Python. Хорошо знаете C++ ...,"Наша команда заботится о том, чтобы пользовате...",От 3 до 6 лет,False,2023-06-01T10:24:01+0300,https://api.hh.ru/vacancies/81397120?host=hh.ru,Полная занятость,81397120,NaN,"[{'name': 'Python'}, {'name': 'Алгоритмы'}, {'...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Наша миссия - сделать поисковую выдачу каче...
365,ML-разработчик в группу исследований машинного...,"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Писать код на C++ и Python. Хорошо знаете C++ ...,"Наша команда заботится о том, чтобы пользовате...",От 3 до 6 лет,False,2023-06-01T10:24:01+0300,https://api.hh.ru/vacancies/81397120?host=hh.ru,Полная занятость,81397120,NaN,"[{'name': 'Python'}, {'name': 'Алгоритмы'}, {'...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Наша миссия - сделать поисковую выдачу каче...
410,ML-разработчик в группу исследований машинного...,"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Писать код на C++ и Python. Хорошо знаете C++ ...,"Наша команда заботится о том, чтобы пользовате...",От 3 до 6 лет,False,2023-06-01T10:24:01+0300,https://api.hh.ru/vacancies/81397120?host=hh.ru,Полная занятость,81397120,NaN,"[{'name': 'Python'}, {'name': 'Алгоритмы'}, {'...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Наша миссия - сделать поисковую выдачу каче...


In [10]:
# Delete the duplicates.
#
# Удалим дубли.  
df = df.drop_duplicates(subset='id')

In [11]:
df.shape

(1839, 20)

### Role column

In [12]:
# Let's see what kind of roles we have.
#
# Посмотрим, какие роли у нас есть.  
df.role.value_counts()

Программист, разработчик                                499
Аналитик                                                409
BI-аналитик, аналитик данных                            277
Дата-сайентист                                          260
Другое                                                  100
Копирайтер, редактор, корректор                          56
Учитель, преподаватель, педагог                          27
DevOps-инженер                                           17
Финансовый аналитик, инвестиционный аналитик             16
Продуктовый аналитик                                     14
Бизнес-аналитик                                          14
Инженер-конструктор, инженер-проектировщик               13
Руководитель отдела аналитики                            13
Руководитель группы разработки                           13
Системный инженер                                        10
Системный аналитик                                       10
Системный администратор                 

In [13]:
# Total unique roles.
#
# Всего уникальных ролей.  
df.role.nunique()

49

In [14]:
df.role.unique()

array(['Дата-сайентист', 'Специалист технической поддержки', 'Аналитик',
       'Другое', 'Программист, разработчик', 'Тестировщик',
       'Руководитель группы разработки', 'Менеджер продукта',
       'Финансовый аналитик, инвестиционный аналитик',
       'Менеджер по продажам, менеджер по работе с клиентами',
       'Бизнес-аналитик', 'Сетевой инженер',
       'BI-аналитик, аналитик данных',
       'Специалист по информационной безопасности',
       'Научный специалист, исследователь',
       'Учитель, преподаватель, педагог',
       'Оператор производственной линии', 'Руководитель проектов',
       'Оператор ПК, оператор базы данных', 'DevOps-инженер',
       'Системный администратор', 'Системный инженер',
       'Системный аналитик', 'Кредитный специалист',
       'Руководитель отдела аналитики', 'Администратор',
       'Инженер по эксплуатации', 'Архитектор', 'Инженер по качеству',
       'Технолог', 'Продуктовый аналитик', 'Методолог', 'Аудитор',
       'Инженер-конструктор, инже

In [15]:
# We see that there are roles in this list that 
# may not seem suitable for us. Let's check them.
#
# Мы видим, что в этом списке есть роли, которые,
# кажется, могут не подойти нам. Проверим их.
del_roles = ['Копирайтер, редактор, корректор',
             'Дизайнер, художник',
             'Переводчик',
             'Журналист, корреспондент',
             'Учитель, преподаватель, педагог']
df[(df.role.isin(del_roles) == True)]

,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description
360,Доцент Департамента анализа данных и машинного...,"Учитель, преподаватель, педагог",Финансовый университет при Правительстве РФ,Москва,90000.0,250000.0,RUR,Профильное базовое физико-математическое или т...,Преподавание дисциплин: Машинное обучение; Ком...,От 1 года до 3 лет,False,2023-06-05T14:02:39+0300,https://api.hh.ru/vacancies/81545268?host=hh.ru,Полная занятость,81545268,True,"[{'name': 'Python'}, {'name': 'Преподаватель'}...","[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>ФГОБУ Финансовый университет при Пр...
382,"Преподаватель программирования (Data Science, ...","Учитель, преподаватель, педагог",Березняк Антон Викторович,Москва,70000.0,NaN,RUR,Ты будешь вести онлайн уроки один на один с уч...,Осуществлять индивидуальный подбор учебных мат...,От 1 года до 3 лет,True,2023-06-14T14:20:21+0300,https://api.hh.ru/vacancies/81711478?host=hh.ru,Частичная занятость,81711478,True,"[{'name': 'Индивидуальное обучение'}, {'name':...",[],"{'id': 'remote', 'name': 'Удаленная работа'}","<p><strong>Привет, наш потенциальный коллега! ..."
897,Assistant Professor - Robotics & Computer Visi...,"Учитель, преподаватель, педагог",Университет Иннополис,Москва,NaN,NaN,NaN,PhD degree (in <highlighttext>Computer</highli...,Up to 120 academic hours of frontal teaching t...,От 1 года до 3 лет,False,2023-06-13T15:43:54+0300,https://api.hh.ru/vacancies/81857336?host=hh.ru,Полная занятость,81857336,NaN,"[{'name': 'Information Technology'}, {'name': ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>The Faculty of Computer Science and Enginee...
902,Assistant Professor - Robotics & Computer Visi...,"Учитель, преподаватель, педагог",Университет Иннополис,Санкт-Петербург,NaN,NaN,NaN,PhD degree (in <highlighttext>Computer</highli...,Up to 120 academic hours of frontal teaching t...,От 3 до 6 лет,False,2023-06-13T15:44:20+0300,https://api.hh.ru/vacancies/81857371?host=hh.ru,Полная занятость,81857371,NaN,"[{'name': 'Information Technology'}, {'name': ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>The Faculty of Computer Science and Enginee...
903,Full Professor - Robotics & Computer Vision (eng),"Учитель, преподаватель, педагог",Университет Иннополис,Санкт-Петербург,NaN,NaN,NaN,...Deep Learning (usage instrument is <highlig...,Up to 120 academic hours of frontal teaching t...,Более 6 лет,False,2023-06-13T15:44:51+0300,https://api.hh.ru/vacancies/81857405?host=hh.ru,Полная занятость,81857405,NaN,"[{'name': 'Information Technology'}, {'name': ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>The Faculty of Computer Science and Enginee...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231,Копирайтер для нейросети (в Яндекс),"Копирайтер, редактор, корректор",Гончаров Никита Дмитриевич,Волгоград,75000.0,NaN,RUR,Высшее образование по одной из следующих специ...,Проверять факты и отличать достоверные источни...,От 1 года до 3 лет,False,2023-06-14T11:44:52+0300,https://api.hh.ru/vacancies/81700117?host=hh.ru,Частичная занятость,81700117,True,"[{'name': 'Редактирование текстов'}, {'name': ...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p><strong>Работа тренером/редактором искусств...
2232,Учитель русского языка и литературы (для нейро...,"Учитель, преподаватель, педагог",Гончаров Никита Дмитриевич,Пермь,75000.0,NaN,RUR,Высшее образование по одной из следующих специ...,Проверять факты и отличать достоверные источни...,От 1 года до 3 лет,False,2023-06-14T11:44:36+0300,https://api.hh.ru/vacancies/81700090?host=hh.ru,Частичная занятость,81700090,True,"[{'name': 'Редактирование текстов'}, {'name': ...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p><strong>Работа тренером/редактором искусств...
2234,"Препо

In [16]:
# These ads do not meet our requirements, we remove them.
#
# Эти объявления не соответствуют нашим требованиям, удаляем их.  
df = df[(df.role.isin(del_roles) == False)]

In [17]:
# Let's look at the role of the analyst.
#
# Посмотрим на роль аналитика.
df[(df.role.isin(['Аналитик','BI-аналитик, аналитик данных'])  == True)]

,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description
2,Junior Data scientist/Machine learning,Аналитик,СБЕР,Москва,NaN,NaN,NaN,Обучение классических моделей машинного обучен...,Анализ и использование различных источников да...,От 1 года до 3 лет,False,2023-06-02T12:01:47+0300,https://api.hh.ru/vacancies/81456638?host=hh.ru,Полная занятость,81456638,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ищем Data scientist в Блок Сервисы. Это Бло...
30,Middle Data scientist/Machine learning,Аналитик,СБЕР,Москва,NaN,NaN,NaN,Обучение классических модели машинного обучени...,Анализ и использование различных источников да...,От 1 года до 3 лет,False,2023-06-02T11:46:24+0300,https://api.hh.ru/vacancies/81455374?host=hh.ru,Полная занятость,81455374,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ищем Data scientist в Блок Сервисы. Это Бло...
48,Data Scientist / ML-инженер,Аналитик,Медиалогия,Москва,NaN,NaN,NaN,Уверенное знание Python и библиотек для работы...,Построение и тестирование гипотез. Создание мо...,От 1 года до 3 лет,False,2023-05-29T11:43:04+0300,https://api.hh.ru/vacancies/81226049?host=hh.ru,Полная занятость,81226049,NaN,"[{'name': 'Python'}, {'name': 'Adobe Photoshop...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Медиалогия разрабатывает высоконагруженные ...
58,Python developer (ML Engineer),Аналитик,СБЕР,Санкт-Петербург,NaN,NaN,NaN,"В поисках Python-разработчика, готового перейт...",Наша команда разрабатывает и поддерживает боле...,От 1 года до 3 лет,False,2023-05-29T10:56:54+0300,https://api.hh.ru/vacancies/81220679?host=hh.ru,Полная занятость,81220679,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Наша команда разрабатывает и поддерживает б...
69,ML-инженер (Банковский скоринг),"BI-аналитик, аналитик данных",Займиго МФК,Москва,150000.0,NaN,RUR,Серьезную математическую подготовку. Опыт прим...,Построение рисковых скоринговых моделей. Монит...,От 1 года до 3 лет,False,2023-05-23T12:36:27+0300,https://api.hh.ru/vacancies/80884767?host=hh.ru,Полная занятость,80884767,False,"[{'name': 'Анализ рисков'}, {'name': 'Аналитич...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Zaymigo – </strong>это современная ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129,Младший аналитик Центра прикладного анализа бо...,Аналитик,Национальный исследовательский Томский государ...,Томск,NaN,NaN,NaN,Высшее/неоконченное профессиональное образован...,Проведение аналитической и научно-исследовател...,От 1 года до 3 лет,False,2023-05-30T05:55:49+0300,https://api.hh.ru/vacancies/81270894?host=hh.ru,Полная занятость,81270894,NaN,"[{'name': 'Статистический анализ'}, {'name': '...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<strong>Обязанности:</strong> <ul> <li> <p>Про...
2136,Аналитик больших данных (направление кибербезо...,Аналитик,Банк СОЮЗ (АО),Москва,200000.0,NaN,RUR,"Знание, понимание и навыки администрирования, ...",Поиск киберугроз в инфраструктуре банка на осн...,От 3 до 6 лет,False,2023-05-28T11:49:42+0300,https://api.hh.ru/vacancies/80696034?host=hh.ru,Полная занятость,80696034,False,"[{'name': 'Object Storage'}, {'name': 'NOSQL с...",[],"{'id': 'fullDay', 'name': 'Полный день'}","Вакансия будет интересна тем, кто устал от адм..."
2140,Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,NaN,Опыт написания технических заданий на разработ...,Участие в создании бизнес-метрик продукта. Пло...,От 3 до 6 лет,False,2023-06-16T11:55:14+0300,https://api.hh.ru/vacancies/81088795?host=hh.ru,Полная занятость,81088795,NaN,[],[],"{'id': 'remote', 'name': 'Удаленная работа'}","<p><strong>Итак, тебе предстоит:</strong></p> ..."
2141,Старший Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,NaN,Опыт работы аналитиком от 2 лет. Развитые комм

Мы видим, что есть много вакансий, не относящихся к Data Science. Отфильтруем их

In [18]:
# Vacancies that have these phrases in the name parameter,
# we do not delete.
#
# Вакансии у которых в параметре name которых есть эти 
# словосочетания, мы не удаляем.
analyst_roles = ['machine learning',
                 'ML engineer',
                 'ML инженер',
                 'ML-инженер',
                 'ML специалист',
                 'ML developer',
                 'ML разработчик',
                 'ML-специалист',
                 'ML-разработчик',
                 'машинное обучение',
                 'машинному обучению',
                 'AI developer',
                 'AI разработчик',
                 'MLops',
                 'ML-ops',
                 'data scientist',
                 'ML ops',
                 'data engineer',
                 'дата инженер',
                 'дата-инженер',
                 'Data-инженер',
                 'Data инженер',
                 'ETL',
                 'DWH',
                 'NLP',
                 'Computer vision'] 

# Vacancies that have these phrases in the 
# snippet.requirement parameter, we do not delete.
#
# Вакансии у которых в параметре snippet.requirement
# которых есть эти словосочетания, мы не удаляем.
analyst_skills = ['Python','машинное обучение','машинного обучения']

# Vacancies that have these phrases in the 
# key_skills parameter, we do not delete.
#
# Вакансии у которых в параметре key_skills
# которых есть эти навыки, мы не удаляем.
analyst_key_skills = ['Python']
analyst_description = ['Python']
df2 = df[(df.name.str.contains('|'.join(analyst_roles), case=False) == False) &
         (df.role.isin(['Аналитик','BI-аналитик, аналитик данных'])  == True) &
         (df['snippet.requirement'].str.contains('|'.join(analyst_skills), case=False) == False) &
         (df.key_skills.str.contains('|'.join(analyst_key_skills), case=False) == False) &
         (df.description.str.contains('|'.join(analyst_description), case=False) == False)]
print(df2.shape)
# Using filters, we got a list of vacancies that need to be deleted.
#
# С помощью фильтров получили список вакансий, которые нужно удалить.
df2

(179, 20)


,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description
343,Бизнес-аналитик на Платформу Машинного обучения,Аналитик,Газпромбанк,Москва,NaN,NaN,NaN,Развитые аналитические навыки. Понимание спец...,Сотрудничать с потенциальными пользователями M...,От 1 года до 3 лет,False,2023-06-15T11:33:28+0300,https://api.hh.ru/vacancies/81949391?host=hh.ru,Полная занятость,81949391,NaN,[],[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>В Газпромбанке нами движет страсть к иннова...
1376,Аналитик данных / Data analyst (junior),Аналитик,КрокусАппс,Минск,400.0,500.0,USD,Высшее экономическое или физико-математическое...,Помощь в сборе данных для отчетов и статистики...,От 1 года до 3 лет,False,2023-06-06T12:55:35+0300,https://api.hh.ru/vacancies/81598271?host=hh.ru,Полная занятость,81598271,False,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>SYNDICATE GROUP</strong></p> <p>Это...
1381,Аналитик данных (Data Analyst),Аналитик,Кыргызкоммерцбанк,Бишкек,NaN,NaN,NaN,Высшее техническое / экономическое образование...,На основании формализованных исходных условий ...,Нет опыта,False,2023-06-14T15:24:39+0300,https://api.hh.ru/vacancies/81913100?host=hh.ru,Полная занятость,81913100,NaN,"[{'name': 'MS Visio'}, {'name': 'Бизнес-анализ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<strong>Обязанности:</strong> <ul> <li>На осно...
1387,Junior Data Analyst,Аналитик,СБЕР,Москва,NaN,NaN,NaN,"Аналитический склад мышления, системный подход...",Анализ бизнес и функциональных требований к ви...,От 1 года до 3 лет,False,2023-05-29T16:36:20+0300,https://api.hh.ru/vacancies/75697706?host=hh.ru,Полная занятость,75697706,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong><em>Задачи</em></strong></p> <ul> <...
1397,Lead BI Analyst / Data Analyst,"BI-аналитик, аналитик данных",Artics Internet Solutions,Москва,NaN,NaN,NaN,Опыт внедрения и сопровождения дашбордов в Mic...,Поддержка и сопровождение дашбордов. Подготовк...,От 3 до 6 лет,False,2023-06-13T13:15:29+0300,https://api.hh.ru/vacancies/81844787?host=hh.ru,Полная занятость,81844787,NaN,"[{'name': 'ClickHouse'}, {'name': 'SQL'}, {'na...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Привет! Мы<strong> Artics Internet Solution...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2076,Аналитик Big data,Аналитик,"Черкизово, Группа предприятий",Москва,NaN,NaN,NaN,"Опыт работы в сфере аналитики продаж, планиров...",Формулирование постановки CR на доработку сист...,От 1 года до 3 лет,False,2023-05-23T10:07:09+0300,https://api.hh.ru/vacancies/79885148?host=hh.ru,Полная занятость,79885148,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Работа в Черкизово — это:</strong><...
2079,Middle Дата Аналитик в продукт МТС Маркетолог ...,Аналитик,МТС,Москва,NaN,NaN,NaN,"Знание SQL на уверенном уровне (join, group by...",Собирать и валидировать данные. Сегментировать...,От 1 года до 3 лет,False,2023-06-09T12:22:59+0300,https://api.hh.ru/vacancies/80228757?host=hh.ru,Полная занятость,80228757,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}","<p><strong>Big Data МТС </strong>– место, где ..."
2088,Руководитель центра монетизации Big Data,"BI-аналитик, аналитик данных",Simble,Москва,NaN,NaN,NaN,Диплом бакалавра и выше в предметной области (...,Проводить маркетинговые исследования для опред...,От 3 до 6 лет,False,2023-05-30T10:24:39+0300,https://api.hh.ru/vacancies/81284767?host=hh.ru,Полная занятость,81284767,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>SIMBLE – это IT-продукт для управле...
2114,Менеджер по внедрению продуктов больших данных...,Аналитик,Tele2,Москва,NaN,NaN,NaN,Высшее техническое или экономическое образован...,Создание новых продуктов Big Data в рамках кон...,От 3 до 6 лет,False,2023-06-13T11:21:20+0300,https://api.hh.ru/vacancies/76

In [19]:
# Delete it
#
# Удаляем
del_adv = df2['id'].tolist()
df = df[(df.id.isin(del_adv) == False)]

In [20]:
df.shape

(1571, 20)

In [21]:
# look at the "Other" role
#
# посмотрим на роль "Другое"
df[(df.role.isin(['Другое'])  == True)]

,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,250000.0,300000.0,RUR,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,True,2023-05-29T16:19:08+0300,https://api.hh.ru/vacancies/81253945?host=hh.ru,Частичная занятость,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Задачи:</strong></p> <ul> <li>Приду...
15,Machine Learning Engineer/MLOps Engineer (Game...,Другое,Kometa.Games,Москва,NaN,NaN,NaN,Опыт работы в Data Science от 2х лет. Опыт раб...,Разработка и внедрение моделей машинного обуче...,От 1 года до 3 лет,False,2023-05-18T18:23:42+0300,https://api.hh.ru/vacancies/80677542?host=hh.ru,Полная занятость,80677542,NaN,"[{'name': 'Python'}, {'name': 'Machine Learnin...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Вам предстоит работать над красочным проект...
29,Machine Learning Project manager,Другое,AX TECHNOLOGY (OOO INSANE GROUP CO),Москва,NaN,NaN,NaN,Опыт работы не менее 3-х лет в роли Project Ma...,"Планирование, координация и контроль выполнени...",От 3 до 6 лет,False,2023-05-25T16:46:59+0300,https://api.hh.ru/vacancies/81050544?host=hh.ru,Полная занятость,81050544,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ax Technology - ведущая компания по разрабо...
33,Machine Learning Project manager,Другое,AX TECHNOLOGY (OOO INSANE GROUP CO),Санкт-Петербург,NaN,NaN,NaN,Опыт работы не менее 3-х лет в роли Project Ma...,"Планирование, координация и контроль выполнени...",От 3 до 6 лет,False,2023-05-25T16:49:24+0300,https://api.hh.ru/vacancies/81050759?host=hh.ru,Полная занятость,81050759,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ax Technology - ведущая компания по разрабо...
34,Machine Learning Project manager,Другое,AX TECHNOLOGY (OOO INSANE GROUP CO),Ташкент,NaN,NaN,NaN,Опыт работы не менее 3-х лет в роли Project Ma...,"Планирование, координация и контроль выполнени...",От 3 до 6 лет,False,2023-05-23T13:32:04+0300,https://api.hh.ru/vacancies/80889356?host=hh.ru,Полная занятость,80889356,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ax Technology - ведущая компания по разрабо...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,Переводчик для нейросети (в Яндекс),Другое,Гончаров Никита Дмитриевич,Екатеринбург,75000.0,NaN,RUR,Высшее образование по одной из следующих специ...,Проверять факты и отличать достоверные источни...,От 1 года до 3 лет,False,2023-06-15T12:21:19+0300,https://api.hh.ru/vacancies/81749000?host=hh.ru,Частичная занятость,81749000,True,"[{'name': 'Редактирование текстов'}, {'name': ...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p><strong>Работа тренером/редактором искусств...
2194,"Преподаватель нейросети (журналистика, написан...",Другое,Гончаров Никита Дмитриевич,Москва,75000.0,NaN,RUR,Высшее образование по одной из следующих специ...,Проверять факты и отличать достоверные источни...,От 1 года до 3 лет,False,2023-06-15T12:21:19+0300,https://api.hh.ru/vacancies/81748998?host=hh.ru,Частичная занятость,81748998,True,"[{'name': 'Редактирование текстов'}, {'name': ...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p><strong>Работа тренером/редактором искусств...
2195,Переводчик для нейросети (в Яндекс),Другое,Гончаров Никита Дмитриевич,Ростов-на-Дону,75000.0,NaN,RUR,Высшее образование по одной из следующих специ...,Проверять факты и отличать достоверные источни...,От 1 года до 3 лет,False,2023-06-15T12:21:19+0300,https://api.hh.ru/vacancies/81749002?host=hh.ru,Частичная занятость,81749002,True,"[{'name': 'Редактирование текстов'}, {'name': ...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p><strong>Работа тренером/редактором искусств...
2196

Let's make a filter for vacancies that need to be deleted

Сделаем фильтр на вакансии, которые нужно удалить

In [22]:
other_roles = ['Machine Learning Engineer',
               'ML инженер',
               'ML - специалист',
               'MLOps',
               'ML OPS',
               'Data Scientist',
               'Data Engineer',
               'Data инженер',
               'Data-инженер',
               'ML Engineer'] 

# Vacancies that have these phrases in the 
# snippet.requirement parameter, we do not delete.
#
# Вакансии у которых в параметре snippet.requirement
# которых есть эти словосочетания, мы не удаляем.
other_skills = ['Python','машинное обучение','машинного обучения']

# Vacancies that have these skills in the
# key_skills parameter, we do not delete.
#
# Вакансии у которых в параметре key_skills
# которых есть эти навыки, мы не удаляем.
other_key_skills = ['Python']
other_description = ['Python']
df3 = df[(df.name.str.contains('|'.join(other_roles), case=False) == False) &
         (df.role.isin(['Другое'])  == True) &
         (df['snippet.requirement'].str.contains('|'.join(other_skills), case=False) == False) &
         (df.key_skills.str.contains('|'.join(other_key_skills), case=False) == False) &
         (df.description.str.contains('|'.join(other_description), case=False) == False)]
print(df3.shape)
df3

(32, 20)


,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description
29,Machine Learning Project manager,Другое,AX TECHNOLOGY (OOO INSANE GROUP CO),Москва,NaN,NaN,NaN,Опыт работы не менее 3-х лет в роли Project Ma...,"Планирование, координация и контроль выполнени...",От 3 до 6 лет,False,2023-05-25T16:46:59+0300,https://api.hh.ru/vacancies/81050544?host=hh.ru,Полная занятость,81050544,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ax Technology - ведущая компания по разрабо...
33,Machine Learning Project manager,Другое,AX TECHNOLOGY (OOO INSANE GROUP CO),Санкт-Петербург,NaN,NaN,NaN,Опыт работы не менее 3-х лет в роли Project Ma...,"Планирование, координация и контроль выполнени...",От 3 до 6 лет,False,2023-05-25T16:49:24+0300,https://api.hh.ru/vacancies/81050759?host=hh.ru,Полная занятость,81050759,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ax Technology - ведущая компания по разрабо...
34,Machine Learning Project manager,Другое,AX TECHNOLOGY (OOO INSANE GROUP CO),Ташкент,NaN,NaN,NaN,Опыт работы не менее 3-х лет в роли Project Ma...,"Планирование, координация и контроль выполнени...",От 3 до 6 лет,False,2023-05-23T13:32:04+0300,https://api.hh.ru/vacancies/80889356?host=hh.ru,Полная занятость,80889356,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ax Technology - ведущая компания по разрабо...
341,Разметчик данных для машинного обучения,Другое,Рокет Проджект,Новосибирск,21000.0,NaN,RUR,"Ты идеально подходишь, если: Ознакомиться с те...",Выполнять задания на специальной платформе. В ...,Нет опыта,True,2023-06-05T18:37:06+0300,https://api.hh.ru/vacancies/81565270?host=hh.ru,Полная занятость,81565270,False,"[{'name': 'Грамотная речь'}, {'name': 'Высокая...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>Компания <strong>Training Data</strong> зан...
344,Оператор печатной машины (обучение),Другое,ОРТО,Самара,55000.0,NaN,RUR,Средне-техническое образование. Опыт работы на...,"Выпуск продукции,соответствующей требованиям к...",Нет опыта,False,2023-05-18T15:08:36+0300,https://api.hh.ru/vacancies/80666520?host=hh.ru,Полная занятость,80666520,False,[],[],"{'id': 'shift', 'name': 'Сменный график'}","<p><strong>В связи с расширением, крупная прои..."
345,Разметчик данных для машинного обучения,Другое,Рокет Проджект,Пермь,21000.0,NaN,RUR,Ознакомиться с техническим заданием и пройти т...,Выполнять задания на специальной платформе. В ...,Нет опыта,True,2023-06-12T16:09:09+0300,https://api.hh.ru/vacancies/81803672?host=hh.ru,Полная занятость,81803672,False,"[{'name': 'Грамотная речь'}, {'name': 'Высокая...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>Компания <strong>Training Data</strong> зан...
346,Разметчик данных для машинного обучения,Другое,Рокет Проджект,Челябинск,21000.0,NaN,RUR,Ознакомиться с техническим заданием и пройти т...,Выполнять задания на специальной платформе. В ...,Нет опыта,True,2023-06-16T12:37:00+0300,https://api.hh.ru/vacancies/82032008?host=hh.ru,Полная занятость,82032008,False,"[{'name': 'Грамотная речь'}, {'name': 'Высокая...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>Компания <strong>Training Data</strong> зан...
347,Разметчик данных для машинного обучения,Другое,Рокет Проджект,Москва,21000.0,NaN,RUR,Ознакомиться с техническим заданием и пройти т...,Выполнять задания на специальной платформе. В ...,Нет опыта,True,2023-06-12T16:50:39+0300,https://api.hh.ru/vacancies/81804627?host=hh.ru,Полная занятость,81804627,False,"[{'name': 'Грамотная речь'}, {'name': 'Высокая...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>Компания <strong>Training Data</strong> зан...
351,Разметчик данных для машинного обучения,Другое,Рокет Проджект,Волгоград,21000.0,NaN,RUR,Ознакомиться с техническим заданием и пройти т...,Выполнять задания на специальной платформе. В ...,Нет опыта,True,2023-06-12T16:09:09+0300

In [23]:
# Delete it.
#
# Удаляем.
del_adv = df3['id'].tolist()
df = df[(df.id.isin(del_adv) == False)]

In [24]:
df.shape

(1539, 20)

Let's see the remaining roles all in total, since there are not many entries.

Оставшиеся роли посмотрим все в сумме, так как записей не много.

In [25]:
df.role.unique()

array(['Дата-сайентист', 'Специалист технической поддержки', 'Аналитик',
       'Другое', 'Программист, разработчик', 'Тестировщик',
       'Руководитель группы разработки', 'Менеджер продукта',
       'Финансовый аналитик, инвестиционный аналитик',
       'Менеджер по продажам, менеджер по работе с клиентами',
       'Бизнес-аналитик', 'Сетевой инженер',
       'BI-аналитик, аналитик данных',
       'Специалист по информационной безопасности',
       'Научный специалист, исследователь',
       'Оператор производственной линии', 'Руководитель проектов',
       'Оператор ПК, оператор базы данных', 'DevOps-инженер',
       'Системный администратор', 'Системный инженер',
       'Системный аналитик', 'Кредитный специалист',
       'Руководитель отдела аналитики', 'Администратор',
       'Инженер по эксплуатации', 'Архитектор', 'Инженер по качеству',
       'Технолог', 'Продуктовый аналитик', 'Методолог', 'Аудитор',
       'Инженер-конструктор, инженер-проектировщик',
       'SMM-менеджер, 

In [26]:
df[(df.role.isin(['Специалист технической поддержки', 'Тестировщик',
       'Руководитель группы разработки', 'Менеджер продукта',
       'Финансовый аналитик, инвестиционный аналитик',
       'Менеджер по продажам, менеджер по работе с клиентами',
       'Бизнес-аналитик', 'Сетевой инженер',
       'Специалист по информационной безопасности',
       'Научный специалист, исследователь',
       'Оператор производственной линии', 'Руководитель проектов',
       'Оператор ПК, оператор базы данных', 'DevOps-инженер',
       'Системный администратор', 'Системный инженер',
       'Системный аналитик', 'Кредитный специалист',
       'Руководитель отдела аналитики', 'Администратор',
       'Инженер по эксплуатации', 'Архитектор', 'Инженер по качеству',
       'Технолог', 'Продуктовый аналитик', 'Методолог', 'Аудитор',
       'Инженер-конструктор, инженер-проектировщик',
       'SMM-менеджер, контент-менеджер', 'Менеджер по персоналу',
       'Казначей', 'Менеджер по закупкам', 'Офис-менеджер',
       'Комплаенс-менеджер', 'Финансовый контролер',
       'Менеджер по маркетингу, интернет-маркетолог',
       'Маркетолог-аналитик', 'Менеджер по логистике, менеджер по ВЭД',
       'Юрисконсульт'])  == True)]

,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description
1,Куратор на курс Start machine learning,Специалист технической поддержки,Карпов Курсес,Санкт-Петербург,50000.0,NaN,RUR,Умеете грамотно выражать свои мысли в письменн...,Модерировать чаты со студентами (до 300 челове...,От 1 года до 3 лет,False,2023-06-13T18:16:41+0300,https://api.hh.ru/vacancies/81866480?host=hh.ru,Полная занятость,81866480,False,"[{'name': 'Грамотность'}, {'name': 'Грамотная ...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>Karpov.courses — молодой EdTech стартап в с...
9,Инженер-тестировщик в команду Machine Learning,Тестировщик,Центр финансовых технологий,Новосибирск,NaN,NaN,NaN,Имеют настроенный CI\CD в gitlab. Уверенным вл...,Кредитный скоринг. Персонализация маркетингово...,От 1 года до 3 лет,False,2023-06-13T06:25:07+0300,https://api.hh.ru/vacancies/71392355?host=hh.ru,Полная занятость,71392355,NaN,"[{'name': 'Python'}, {'name': 'Git'}, {'name':...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>ГК «Центр Финансовых Технологий» - провайде...
10,Team Lead Machine Learning (Computer Vision),Руководитель группы разработки,EKSLi,Москва,250000.0,NaN,RUR,Опыт работы в роли Team Lead от 1 года. Опыт р...,Проектирование систем машинного зрения для сор...,От 1 года до 3 лет,False,2023-06-16T14:06:31+0300,https://api.hh.ru/vacancies/82040145?host=hh.ru,Полная занятость,82040145,False,[],"[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Лидер команды разработки системы </...
19,Product manager (machine learning),Менеджер продукта,LC Group,Санкт-Петербург,NaN,NaN,NaN,Профильное математическое образование. Не мене...,Сбор и обработка данных с помощью SQL и Python...,От 3 до 6 лет,False,2023-06-13T07:10:24+0300,https://api.hh.ru/vacancies/78971105?host=hh.ru,Полная занятость,78971105,NaN,"[{'name': 'Python'}, {'name': 'Статистический ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Мы сохранили в себе важный плюс от стартапа...
26,Product manager (machine learning),Менеджер продукта,LC Group,Екатеринбург,NaN,NaN,NaN,Профильное математическое образование. Не мене...,Сбор и обработка данных с помощью SQL и Python...,От 3 до 6 лет,False,2023-06-13T07:10:21+0300,https://api.hh.ru/vacancies/78971108?host=hh.ru,Полная занятость,78971108,NaN,"[{'name': 'Python'}, {'name': 'Статистический ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Мы сохранили в себе важный плюс от стартапа...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,Менеджер по развитию бизнеса B2B (монетизация ...,"Менеджер по продажам, менеджер по работе с кли...",Платформа ОФД,Москва,100000.0,150000.0,RUR,Имеешь успешный опыт в продажах банковских/фин...,Развитие направления бизнеса компании – аналит...,От 1 года до 3 лет,False,2023-06-14T16:32:09+0300,https://api.hh.ru/vacancies/81476750?host=hh.ru,Полная занятость,81476750,False,[{'name': 'B2B Продажи'}],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Платформа ОФД </strong>- аккредитов...
2147,Автор постов про нейросети и ИИ,"SMM-менеджер, контент-менеджер",Логомашина,Санкт-Петербург,30000.0,50000.0,RUR,Сообразительность и креативность. Ответственно...,"Мне нужны авторы контента, которые увлекаются ...",От 1 года до 3 лет,False,2023-05-24T09:37:16+0300,https://api.hh.ru/vacancies/80929623?host=hh.ru,Полная занятость,80929623,False,"[{'name': 'SMM'}, {'name': 'Копирайтинг'}, {'n...",[],"{'id': 'fullDay', 'name': 'Полный день'}","<p>Привет! Я Николь, автор контента из Логомаш..."
2151,Специалист по нейросетям (мониторинг и тестиро...,Маркетолог-аналитик,MSM Digital,Санкт-Петербург,60000.0,NaN,RUR,"Глубокий интерес, подтвержденный навыками и те...",Отслеживать новые глобальные тренды уже извест...,Нет опыта,True,2023-05-28T17:16:05+0300,https:/

In [27]:
other2_roles = ['Machine Learning Engineer',
                'ML инженер',
                'ML - специалист',
                'MLOps',
                'ML OPS',
                'Data Scientist',
                'Data Engineer',
                'Data инженер',
                'Data-инженер',
                'Data-engineer',
                'Дата инженер',
                'ML Engineer',
                'Developer']  

# Vacancies that have these phrases in the
# snippet.requirement parameter, we do not delete.
#
# Вакансии у которых в параметре snippet.requirement
# которых есть эти словосочетания, мы не удаляем.
other2_skills = ['Python','машинное обучение','машинного обучения']

# Vacancies that have these skills in the key_skills
# parameter, we do not delete.
#
# Вакансии у которых в параметре key_skills
# которых есть эти навыки, мы не удаляем.
other2_key_skills = ['Python']
other2_description = ['Python']
df4 = df[(df.name.str.contains('|'.join(other2_roles), case=False) == False) &
         (df.role.isin(['Специалист технической поддержки','Тестировщик',
       'Руководитель группы разработки', 'Менеджер продукта',
       'Финансовый аналитик, инвестиционный аналитик',
       'Менеджер по продажам, менеджер по работе с клиентами',
       'Бизнес-аналитик', 'Сетевой инженер',
       'Специалист по информационной безопасности',
       'Научный специалист, исследователь',
       'Оператор производственной линии', 'Руководитель проектов',
       'Оператор ПК, оператор базы данных', 'DevOps-инженер',
       'Системный администратор', 'Системный инженер',
       'Системный аналитик', 'Кредитный специалист',
       'Руководитель отдела аналитики', 'Администратор',
       'Инженер по эксплуатации', 'Архитектор', 'Инженер по качеству',
       'Технолог', 'Продуктовый аналитик', 'Методолог', 'Аудитор',
       'Инженер-конструктор, инженер-проектировщик',
       'SMM-менеджер, контент-менеджер', 'Менеджер по персоналу',
       'Казначей', 'Менеджер по закупкам', 'Офис-менеджер',
       'Комплаенс-менеджер', 'Финансовый контролер',
       'Менеджер по маркетингу, интернет-маркетолог',
       'Маркетолог-аналитик', 'Менеджер по логистике, менеджер по ВЭД',
       'Юрисконсульт'])  == True) &
       (df['snippet.requirement'].str.contains('|'.join(other2_skills), case=False) == False) &
       (df.key_skills.str.contains('|'.join(other2_key_skills), case=False) == False) &
       (df.description.str.contains('|'.join(other2_description), case=False) == False)]
print(df4.shape)
df4

(57, 20)


,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description
1,Куратор на курс Start machine learning,Специалист технической поддержки,Карпов Курсес,Санкт-Петербург,50000.0,NaN,RUR,Умеете грамотно выражать свои мысли в письменн...,Модерировать чаты со студентами (до 300 челове...,От 1 года до 3 лет,False,2023-06-13T18:16:41+0300,https://api.hh.ru/vacancies/81866480?host=hh.ru,Полная занятость,81866480,False,"[{'name': 'Грамотность'}, {'name': 'Грамотная ...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>Karpov.courses — молодой EdTech стартап в с...
32,"Менеджер по развитию бизнеса / BDM (BI,Data Go...","Менеджер по продажам, менеджер по работе с кли...",КОРУС Консалтинг,Москва,NaN,NaN,NaN,"Погружение"" в бизнес заказчика, готовность ори...","Обеспечение выполнения квартальных, годовых и ...",От 3 до 6 лет,False,2023-06-06T10:54:15+0300,https://api.hh.ru/vacancies/81587373?host=hh.ru,Полная занятость,81587373,NaN,"[{'name': 'B2B Продажи'}, {'name': 'Ведение пе...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>КОРУС Консалтинг </strong>– российс...
37,"Solution Architect / Presales Expert (BI,Data ...",Бизнес-аналитик,КОРУС Консалтинг,Москва,NaN,NaN,NaN,Навыки проектирования высоконагруженных и отка...,Проработка архитектур комплексных решений в ча...,От 3 до 6 лет,False,2023-06-14T13:16:33+0300,https://api.hh.ru/vacancies/81902534?host=hh.ru,Полная занятость,81902534,NaN,"[{'name': 'Бизнес-анализ'}, {'name': 'Оптимиза...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>КОРУС Консалтинг </strong>– российс...
362,Оператор на упаковочную машину (обучение),Оператор производственной линии,КАРАВАЙ,Шелехов,42000.0,NaN,RUR,Ответственность. Умение работать с техникой.,Упаковка готовой продукции. Настройка упаковоч...,Нет опыта,False,2023-06-13T03:26:25+0300,https://api.hh.ru/vacancies/81194425?host=hh.ru,Полная занятость,81194425,False,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<strong>Обязанности:</strong> <ul> <li>Упаковк...
371,Оператор Машинного обучения,"Оператор ПК, оператор базы данных",СБЕР,Орск,26000.0,NaN,RUR,Специалист со средним профессиональным/неполны...,"Обработка текстовых, голосовых, графических/ви...",Нет опыта,False,2023-06-15T09:38:21+0300,https://api.hh.ru/vacancies/79874053?host=hh.ru,Полная занятость,79874053,False,[{'name': 'Работа с большим объемом информации...,[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Сбер — лучший работодатель России. У нас бо...
1451,Data Analyst,Тестировщик,Перфект Системс,Москва,100000.0,NaN,RUR,"Уверенное знание SQL, на уровне написания запр...",Разработка сценариев тестирования интеграционн...,От 1 года до 3 лет,False,2023-06-15T15:32:59+0300,https://api.hh.ru/vacancies/80512225?host=hh.ru,Полная занятость,80512225,False,"[{'name': 'Atlassian Jira'}, {'name': 'DWH'}, ...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p><strong>Добрый день!</strong><br /><br />Ко...
1497,"Бизнес-аналитик/Data аналитик ( Управление ""Ус...",Бизнес-аналитик,Банк ВТБ (ПАО),Москва,NaN,NaN,NaN,Опыт построения финансовых моделей. Уверенное ...,Построение и контроль ТЭО Проектов Стрима. Эко...,От 1 года до 3 лет,False,2023-05-23T14:23:45+0300,https://api.hh.ru/vacancies/80010379?host=hh.ru,Полная занятость,80010379,NaN,"[{'name': 'SQL'}, {'name': 'Hadoop'}, {'name':...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Обязанности:</strong></p> <ul> <li>...
1544,Business & Data Analyst MEGA Khimki,Бизнес-аналитик,Ингка Россия,Москва,NaN,NaN,NaN,"Minimum 3 years’ experience in data analysis, ...","Provide relevant, user-friendly business infor...",От 3 до 6 лет,False,2023-05-26T17:08:23+0300,https://api.hh.ru/vacancies/80884872?host=hh.ru,Полная занятость,80884872,NaN,[],"[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'fullDay', 'name': 'Полный день'}",<strong>What you will

In [28]:
# Delete it.
#
# Удаляем.
del_adv = df4['id'].tolist()
df = df[(df.id.isin(del_adv) == False)]

In [29]:
df

,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description
0,Machine Learning Engineer,Дата-сайентист,ДОМ.РФ,Москва,NaN,NaN,NaN,Решать амбициозные задачи из области NLP. Увер...,Принимать технические и архитектурные решения ...,Нет опыта,False,2023-06-07T11:25:50+0300,https://api.hh.ru/vacancies/81645171?host=hh.ru,Полная занятость,81645171,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>ЧЕМ НЕОБХОДИМО ЗАНИМАТЬСЯ:</strong>...
2,Junior Data scientist/Machine learning,Аналитик,СБЕР,Москва,NaN,NaN,NaN,Обучение классических моделей машинного обучен...,Анализ и использование различных источников да...,От 1 года до 3 лет,False,2023-06-02T12:01:47+0300,https://api.hh.ru/vacancies/81456638?host=hh.ru,Полная занятость,81456638,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ищем Data scientist в Блок Сервисы. Это Бло...
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,250000.0,300000.0,RUR,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,True,2023-05-29T16:19:08+0300,https://api.hh.ru/vacancies/81253945?host=hh.ru,Частичная занятость,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Задачи:</strong></p> <ul> <li>Приду...
4,Стажер-разработчик (Machine Learning),"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Знание классических алгоритмов и структур данн...,Решение сложных задач для улучшения жизни людей.,Нет опыта,False,2023-05-26T12:10:38+0300,https://api.hh.ru/vacancies/81090082?host=hh.ru,Стажировка,81090082,NaN,[{'name': 'Machine Learning'}],[],"{'id': 'flexible', 'name': 'Гибкий график'}",<p>Машинное обучение и нейронные сети использу...
5,Machine Learning Engineer,"Программист, разработчик",НеоБИТ,Санкт-Петербург,NaN,NaN,NaN,"Знания Мат. аппарата (мат. статистики, теории ...",Исследование и реализация методов предобработк...,От 1 года до 3 лет,False,2023-05-24T13:09:51+0300,https://api.hh.ru/vacancies/80948681?host=hh.ru,Полная занятость,80948681,NaN,"[{'name': 'Machine Learning'}, {'name': 'Compu...",[],"{'id': 'fullDay', 'name': 'Полный день'}","<p>Приглашаем в команду специалиста по ML,кото..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,NaN,Опыт написания технических заданий на разработ...,Участие в создании бизнес-метрик продукта. Пло...,От 3 до 6 лет,False,2023-06-16T11:55:14+0300,https://api.hh.ru/vacancies/81088795?host=hh.ru,Полная занятость,81088795,NaN,[],[],"{'id': 'remote', 'name': 'Удаленная работа'}","<p><strong>Итак, тебе предстоит:</strong></p> ..."
2141,Старший Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,NaN,Опыт работы аналитиком от 2 лет. Развитые комм...,"Сбор, анализ и разработка (уточнение и детализ...",От 1 года до 3 лет,False,2023-06-16T11:43:54+0300,https://api.hh.ru/vacancies/78357091?host=hh.ru,Полная занятость,78357091,NaN,"[{'name': 'Python'}, {'name': 'Spark'}, {'name...",[],"{'id': 'fullDay', 'name': 'Полный день'}","<p>Создание и развитие цифровых инструментов, ..."
2142,Эксперт по исследованиям больших данных,"BI-аналитик, аналитик данных",UNITEL LLC,Ташкент,NaN,NaN,NaN,Опыт 3+ года. Master's or Ph.D в области инфор...,"Оценка целей бизнес задач, бизнес метрик, алго...",От 3 до 6 лет,False,2023-06-02T12:36:09+0300,https://api.hh.ru/vacancies/81459076?host=hh.ru,Полная занятость,81459076,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Обязанности:</strong></p> <ul> <li>...
2233,Разработчик в отдел обработки и анализа данных...,"Программист, разработчик",СКБ МО РФ,Санкт-Петербург,NaN,NaN,NaN,Опыт программирования на языке C++ (17/20). Оп...,Разработка новых и поддержка существующих 

### Create a new attribute 'profession' based on the 'role' and 'name' columns

In [30]:
df['common_role'] = df['name'].astype (str) + ' ' +  df['role'].astype (str)

In [31]:
professions_ds = ['Machine Learning Engineer',
                  'ML инженер',
                  'ML - специалист',
                  'Data Scientist',
                  'ML Engineer',
                  'Дата-сайентист',
                 'ML-инженер',
                 'ML-разработчик',
                 'ML Developer',
                 'ML Разработчик',
                 'Computer Vision Engineer',
                 'Computer-Vision Инженер',
                 'Team Lead Machine Learning',
                 'Специалист по ML',
                 'Machine Learning Team Lead',
                 'ML-Engineer',
                 'ML - инженер',
                 'ML-специалист',
                 'NLP Engineer',
                 'AI Developer',
                 'Python developer (NLP)',
                 'NLP-разработчик',
                 'Data Sientist',
                 'разработчик Computer vision',
                 'Разработчик алгоритмов компьютерного зрения',
                 'Computer Vision инженер',
                 'Developer Computer vision',
                 'TechLead Research Engineer',
                 'Руководитель группы компьютерного зрения',
                 'Стажер направления "Машинное обучение и анализ данных"',
                 'data science engineer',
                 'Специалист по компьютерному зрению',
                 'Computer vision researcher',                                                                      
                 'Computer vision специалист',
                 'NLP Researcher в Sber AI Lab'] 

df5 = df[(df.common_role.str.contains('|'.join(professions_ds), case=False) == True)]
print(df5.shape)
df5

(587, 21)


,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,...,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description,common_role
0,Machine Learning Engineer,Дата-сайентист,ДОМ.РФ,Москва,NaN,NaN,NaN,Решать амбициозные задачи из области NLP. Увер...,Принимать технические и архитектурные решения ...,Нет опыта,...,2023-06-07T11:25:50+0300,https://api.hh.ru/vacancies/81645171?host=hh.ru,Полная занятость,81645171,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>ЧЕМ НЕОБХОДИМО ЗАНИМАТЬСЯ:</strong>...,Machine Learning Engineer Дата-сайентист
2,Junior Data scientist/Machine learning,Аналитик,СБЕР,Москва,NaN,NaN,NaN,Обучение классических моделей машинного обучен...,Анализ и использование различных источников да...,От 1 года до 3 лет,...,2023-06-02T12:01:47+0300,https://api.hh.ru/vacancies/81456638?host=hh.ru,Полная занятость,81456638,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ищем Data scientist в Блок Сервисы. Это Бло...,Junior Data scientist/Machine learning Аналитик
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,250000.0,300000.0,RUR,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,...,2023-05-29T16:19:08+0300,https://api.hh.ru/vacancies/81253945?host=hh.ru,Частичная занятость,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Задачи:</strong></p> <ul> <li>Приду...,Machine Learning Engineer (ML) Другое
5,Machine Learning Engineer,"Программист, разработчик",НеоБИТ,Санкт-Петербург,NaN,NaN,NaN,"Знания Мат. аппарата (мат. статистики, теории ...",Исследование и реализация методов предобработк...,От 1 года до 3 лет,...,2023-05-24T13:09:51+0300,https://api.hh.ru/vacancies/80948681?host=hh.ru,Полная занятость,80948681,NaN,"[{'name': 'Machine Learning'}, {'name': 'Compu...",[],"{'id': 'fullDay', 'name': 'Полный день'}","<p>Приглашаем в команду специалиста по ML,кото...","Machine Learning Engineer Программист, разрабо..."
6,Data scientist / Machine learning engineer (ко...,"Программист, разработчик",VK,Санкт-Петербург,NaN,NaN,NaN,Математически формулировать бизнес-задачи. С п...,Использовать огромное количество разных данных...,От 1 года до 3 лет,...,2023-06-13T12:29:23+0300,https://api.hh.ru/vacancies/81833434?host=hh.ru,Полная занятость,81833434,NaN,"[{'name': 'Python'}, {'name': 'Machine Learnin...",[],"{'id': 'flexible', 'name': 'Гибкий график'}",<p>Мы применяем алгоритмы машинного обучения к...,Data scientist / Machine learning engineer (ко...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1700,Младший аналитик данных,Дата-сайентист,"EGAR TECHNOLOGY, INC.",Москва,NaN,NaN,NaN,"Высшее техническое образование (математика, фи...",Анализ <highlighttext>данных</highlighttext> ф...,От 1 года до 3 лет,...,2023-06-06T15:54:59+0300,https://api.hh.ru/vacancies/80679724?host=hh.ru,Полная занятость,80679724,NaN,"[{'name': 'Python'}, {'name': 'MATLAB'}, {'nam...","[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'remote', 'name': 'Удаленная работа'}",<p><strong>EGAR Technology</strong> - российск...,Младший аналитик данных Дата-сайентист
1790,Аналитик данных,Дата-сайентист,Газпромбанк,Москва,NaN,NaN,NaN,"Знание мат статистики, основных показателей ка...",Чем предстоит заниматься? <highlighttext>Анали...,От 3 до 6 лет,...,2023-06-08T18:23:42+0300,https://api.hh.ru/vacancies/81724524?host=hh.ru,Полная занятость,81724524,NaN,[],[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>Управление розничного моделирования Газпром...,Аналитик данных Дата-сайентист
1805,Аналитик данных,Дата-сайентист,Альфа Телеком (MEGA),Бишкек,NaN,NaN,NaN,Знание и опыт работы с СУБД Oracle. Знания сов...,"Сбор, обработка и анализ информации. Обработка...",От 1 года до 3 лет,...,2023-05-19T15:20:06+0300,https://api.hh.ru/vacancies/80731028?host=hh.ru,П

In [32]:
df.loc[(df.common_role.str.contains('|'.join(professions_ds), case=False) == True),
       'profession'] = 'Data Scientist/ML developer'

In [33]:
# Here we will not filter out those entries that
# have been assigned professions above.
# Let's assume that if the title contains both
# ML-engineer and MLOps, then the profession is MLOps.
#
# Здесь мы не будем отфильтровывать те записи,
# которым присвоили профессии выше.
# Будем считать что если в названии есть и ML-инженер
# и MLOps, то профессия - MLOps.
professions_mlops = ['MLOps',
                     'ML OPS',] 
df6 = df[(df.common_role.str.contains('|'.join(professions_mlops), case=False) == False)]
print(df6.shape)
df6

(1440, 22)


,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,...,url,employment.name,id,salary.gross,key_skills,languages,schedule,description,common_role,profession
0,Machine Learning Engineer,Дата-сайентист,ДОМ.РФ,Москва,NaN,NaN,NaN,Решать амбициозные задачи из области NLP. Увер...,Принимать технические и архитектурные решения ...,Нет опыта,...,https://api.hh.ru/vacancies/81645171?host=hh.ru,Полная занятость,81645171,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>ЧЕМ НЕОБХОДИМО ЗАНИМАТЬСЯ:</strong>...,Machine Learning Engineer Дата-сайентист,Data Scientist/ML developer
2,Junior Data scientist/Machine learning,Аналитик,СБЕР,Москва,NaN,NaN,NaN,Обучение классических моделей машинного обучен...,Анализ и использование различных источников да...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81456638?host=hh.ru,Полная занятость,81456638,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ищем Data scientist в Блок Сервисы. Это Бло...,Junior Data scientist/Machine learning Аналитик,Data Scientist/ML developer
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,250000.0,300000.0,RUR,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81253945?host=hh.ru,Частичная занятость,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Задачи:</strong></p> <ul> <li>Приду...,Machine Learning Engineer (ML) Другое,Data Scientist/ML developer
4,Стажер-разработчик (Machine Learning),"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Знание классических алгоритмов и структур данн...,Решение сложных задач для улучшения жизни людей.,Нет опыта,...,https://api.hh.ru/vacancies/81090082?host=hh.ru,Стажировка,81090082,NaN,[{'name': 'Machine Learning'}],[],"{'id': 'flexible', 'name': 'Гибкий график'}",<p>Машинное обучение и нейронные сети использу...,Стажер-разработчик (Machine Learning) Программ...,NaN
5,Machine Learning Engineer,"Программист, разработчик",НеоБИТ,Санкт-Петербург,NaN,NaN,NaN,"Знания Мат. аппарата (мат. статистики, теории ...",Исследование и реализация методов предобработк...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/80948681?host=hh.ru,Полная занятость,80948681,NaN,"[{'name': 'Machine Learning'}, {'name': 'Compu...",[],"{'id': 'fullDay', 'name': 'Полный день'}","<p>Приглашаем в команду специалиста по ML,кото...","Machine Learning Engineer Программист, разрабо...",Data Scientist/ML developer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,NaN,Опыт написания технических заданий на разработ...,Участие в создании бизнес-метрик продукта. Пло...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/81088795?host=hh.ru,Полная занятость,81088795,NaN,[],[],"{'id': 'remote', 'name': 'Удаленная работа'}","<p><strong>Итак, тебе предстоит:</strong></p> ...","Аналитик больших данных BI-аналитик, аналитик ...",NaN
2141,Старший Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,NaN,Опыт работы аналитиком от 2 лет. Развитые комм...,"Сбор, анализ и разработка (уточнение и детализ...",От 1 года до 3 лет,...,https://api.hh.ru/vacancies/78357091?host=hh.ru,Полная занятость,78357091,NaN,"[{'name': 'Python'}, {'name': 'Spark'}, {'name...",[],"{'id': 'fullDay', 'name': 'Полный день'}","<p>Создание и развитие цифровых инструментов, ...","Старший Аналитик больших данных BI-аналитик, а...",NaN
2142,Эксперт по исследованиям больших данных,"BI-аналитик, аналитик данных",UNITEL LLC,Ташкент,NaN,NaN,NaN,Опыт 3+ года. Master's or Ph.D в области инфор...,"Оценка целей бизнес задач, бизнес метрик, алго...",От 3 до 6 лет,...,https://api.hh.ru/vacancies/81459076?host=hh.ru,Полная занятость,81459076,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Обязанно

In [34]:
df.loc[(df.common_role.str.contains('|'.join(professions_mlops), case=False) == True),
       'profession'] = 'MLOps'

In [35]:
professions_engineer = ['Data Engineer',
                       'Data-инженер',
                       'Data-engineer',
                       'DATA инженер',
                       'Дата инженер',
                       'Тимлид команды дата-инженеров',
                       'Дата-инженер',
                       'Дата - инженер'] 
df7 = df[(df.common_role.str.contains('|'.join(professions_engineer), case=False) == False) &
         (df.profession.isnull()==True)]
print(df7.shape)
df7

(434, 22)


,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,...,url,employment.name,id,salary.gross,key_skills,languages,schedule,description,common_role,profession
4,Стажер-разработчик (Machine Learning),"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Знание классических алгоритмов и структур данн...,Решение сложных задач для улучшения жизни людей.,Нет опыта,...,https://api.hh.ru/vacancies/81090082?host=hh.ru,Стажировка,81090082,NaN,[{'name': 'Machine Learning'}],[],"{'id': 'flexible', 'name': 'Гибкий график'}",<p>Машинное обучение и нейронные сети использу...,Стажер-разработчик (Machine Learning) Программ...,NaN
9,Инженер-тестировщик в команду Machine Learning,Тестировщик,Центр финансовых технологий,Новосибирск,NaN,NaN,NaN,Имеют настроенный CI\CD в gitlab. Уверенным вл...,Кредитный скоринг. Персонализация маркетингово...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/71392355?host=hh.ru,Полная занятость,71392355,NaN,"[{'name': 'Python'}, {'name': 'Git'}, {'name':...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>ГК «Центр Финансовых Технологий» - провайде...,Инженер-тестировщик в команду Machine Learning...,NaN
12,Machine Learning,"Программист, разработчик",СБЕР,Москва,NaN,NaN,NaN,Хорошие знания Python и умение писать качестве...,Участие в командной разработке и доработки бан...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/79927246?host=hh.ru,Полная занятость,79927246,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>В новую R&amp;D AI команду ищем разработчик...,"Machine Learning Программист, разработчик",NaN
19,Product manager (machine learning),Менеджер продукта,LC Group,Санкт-Петербург,NaN,NaN,NaN,Профильное математическое образование. Не мене...,Сбор и обработка данных с помощью SQL и Python...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/78971105?host=hh.ru,Полная занятость,78971105,NaN,"[{'name': 'Python'}, {'name': 'Статистический ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Мы сохранили в себе важный плюс от стартапа...,Product manager (machine learning) Менеджер пр...,NaN
26,Product manager (machine learning),Менеджер продукта,LC Group,Екатеринбург,NaN,NaN,NaN,Профильное математическое образование. Не мене...,Сбор и обработка данных с помощью SQL и Python...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/78971108?host=hh.ru,Полная занятость,78971108,NaN,"[{'name': 'Python'}, {'name': 'Статистический ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Мы сохранили в себе важный плюс от стартапа...,Product manager (machine learning) Менеджер пр...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,Главный программист отдела по обработке и визу...,"Программист, разработчик",ГКУ ТО Центр информационных технологий Тюменск...,Тюмень,50000.0,NaN,RUR,"Высшее образование в сфере IT приветствуется, ...",Разработка новых и модернизация существующих м...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81954209?host=hh.ru,Полная занятость,81954209,False,"[{'name': 'SQL'}, {'name': 'PostgreSQL'}, {'na...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Обязанности:</strong></p> <p>* Разр...,Главный программист отдела по обработке и визу...,NaN
2140,Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,NaN,Опыт написания технических заданий на разработ...,Участие в создании бизнес-метрик продукта. Пло...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/81088795?host=hh.ru,Полная занятость,81088795,NaN,[],[],"{'id': 'remote', 'name': 'Удаленная работа'}","<p><strong>Итак, тебе предстоит:</strong></p> ...","Аналитик больших данных BI-аналитик, аналитик ...",NaN
2141,Старший Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,NaN,Опыт работы аналитиком от 2 лет. Развитые комм...,"Сбор, анализ и разработка (уточнение и детализ...",От 1 года до 3 лет,...,https://api.hh.ru/vacancies/78357091?host=hh.ru,Полная занятость,783570

In [36]:
df.loc[(df.common_role.str.contains('|'.join(professions_engineer), case=False) == True)
       & (df.profession.isnull()==True), 'profession'] = 'Data engineer'

In [37]:
professions_analyst = ['Data Analyst',
                      'Data Аналитик',
                      'Data-аналитик',
                      'Аналитик данных',
                      'data-analyst',
                      'Аналитик (Дата-инженер)',
                      'Дата Аналитик',
                      'Руководитель группы аналитики данных',
                      'Аналитик Data Science'] 
df7 = df[(df.common_role.str.contains('|'.join(professions_analyst), case=False) == False)& (df.profession.isnull()==True)]
print(df7.shape)
df7

(90, 22)


,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,...,url,employment.name,id,salary.gross,key_skills,languages,schedule,description,common_role,profession
4,Стажер-разработчик (Machine Learning),"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Знание классических алгоритмов и структур данн...,Решение сложных задач для улучшения жизни людей.,Нет опыта,...,https://api.hh.ru/vacancies/81090082?host=hh.ru,Стажировка,81090082,NaN,[{'name': 'Machine Learning'}],[],"{'id': 'flexible', 'name': 'Гибкий график'}",<p>Машинное обучение и нейронные сети использу...,Стажер-разработчик (Machine Learning) Программ...,NaN
9,Инженер-тестировщик в команду Machine Learning,Тестировщик,Центр финансовых технологий,Новосибирск,NaN,NaN,NaN,Имеют настроенный CI\CD в gitlab. Уверенным вл...,Кредитный скоринг. Персонализация маркетингово...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/71392355?host=hh.ru,Полная занятость,71392355,NaN,"[{'name': 'Python'}, {'name': 'Git'}, {'name':...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>ГК «Центр Финансовых Технологий» - провайде...,Инженер-тестировщик в команду Machine Learning...,NaN
12,Machine Learning,"Программист, разработчик",СБЕР,Москва,NaN,NaN,NaN,Хорошие знания Python и умение писать качестве...,Участие в командной разработке и доработки бан...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/79927246?host=hh.ru,Полная занятость,79927246,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>В новую R&amp;D AI команду ищем разработчик...,"Machine Learning Программист, разработчик",NaN
19,Product manager (machine learning),Менеджер продукта,LC Group,Санкт-Петербург,NaN,NaN,NaN,Профильное математическое образование. Не мене...,Сбор и обработка данных с помощью SQL и Python...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/78971105?host=hh.ru,Полная занятость,78971105,NaN,"[{'name': 'Python'}, {'name': 'Статистический ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Мы сохранили в себе важный плюс от стартапа...,Product manager (machine learning) Менеджер пр...,NaN
26,Product manager (machine learning),Менеджер продукта,LC Group,Екатеринбург,NaN,NaN,NaN,Профильное математическое образование. Не мене...,Сбор и обработка данных с помощью SQL и Python...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/78971108?host=hh.ru,Полная занятость,78971108,NaN,"[{'name': 'Python'}, {'name': 'Статистический ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Мы сохранили в себе важный плюс от стартапа...,Product manager (machine learning) Менеджер пр...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129,Младший аналитик Центра прикладного анализа бо...,Аналитик,Национальный исследовательский Томский государ...,Томск,NaN,NaN,NaN,Высшее/неоконченное профессиональное образован...,Проведение аналитической и научно-исследовател...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81270894?host=hh.ru,Полная занятость,81270894,NaN,"[{'name': 'Статистический анализ'}, {'name': '...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<strong>Обязанности:</strong> <ul> <li> <p>Про...,Младший аналитик Центра прикладного анализа бо...,NaN
2134,Научный сотрудник (постдок) в НУЛ методов анал...,"Научный специалист, исследователь",Национальный исследовательский университет Выс...,Москва,NaN,NaN,NaN,"Учёную степень по науке о данных, статистике и...",Проводить исследования в одной или нескольких ...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81409175?host=hh.ru,Полная занятость,81409175,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}","<p>Мы, <strong>Высшая школа экономики</strong>...",Научный сотрудник (постдок) в НУЛ методов анал...,NaN
2135,Начальник отдела по обработке и визуализации п...,Архитектор,ГКУ ТО Центр информационных технологий Тюменск...,Тюмень,70000.0,NaN,RUR,Высшее образование в области IT. Опыт работы в...,Руководство отделом. Декомпозиция поступающих ...,От 

In [38]:
df.loc[(df.common_role.str.contains('|'.join(professions_analyst), case=False) == True)
       & (df.profession.isnull()==True), 'profession'] = 'Data analyst'

In [39]:
df.profession.value_counts()

Data Scientist/ML developer    577
Data engineer                  429
Data analyst                   344
MLOps                           42
Name: profession, dtype: int64

In [40]:
# The remaining vacancies are difficult to attribute
# to any category, so we will delete them.
#
# Оставшиеся вакансии сложно отнести к какой-либо 
# категории, поэтому удалим их.
df[(df.profession.isnull()==True)]

,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,...,url,employment.name,id,salary.gross,key_skills,languages,schedule,description,common_role,profession
4,Стажер-разработчик (Machine Learning),"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Знание классических алгоритмов и структур данн...,Решение сложных задач для улучшения жизни людей.,Нет опыта,...,https://api.hh.ru/vacancies/81090082?host=hh.ru,Стажировка,81090082,NaN,[{'name': 'Machine Learning'}],[],"{'id': 'flexible', 'name': 'Гибкий график'}",<p>Машинное обучение и нейронные сети использу...,Стажер-разработчик (Machine Learning) Программ...,NaN
9,Инженер-тестировщик в команду Machine Learning,Тестировщик,Центр финансовых технологий,Новосибирск,NaN,NaN,NaN,Имеют настроенный CI\CD в gitlab. Уверенным вл...,Кредитный скоринг. Персонализация маркетингово...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/71392355?host=hh.ru,Полная занятость,71392355,NaN,"[{'name': 'Python'}, {'name': 'Git'}, {'name':...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>ГК «Центр Финансовых Технологий» - провайде...,Инженер-тестировщик в команду Machine Learning...,NaN
12,Machine Learning,"Программист, разработчик",СБЕР,Москва,NaN,NaN,NaN,Хорошие знания Python и умение писать качестве...,Участие в командной разработке и доработки бан...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/79927246?host=hh.ru,Полная занятость,79927246,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>В новую R&amp;D AI команду ищем разработчик...,"Machine Learning Программист, разработчик",NaN
19,Product manager (machine learning),Менеджер продукта,LC Group,Санкт-Петербург,NaN,NaN,NaN,Профильное математическое образование. Не мене...,Сбор и обработка данных с помощью SQL и Python...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/78971105?host=hh.ru,Полная занятость,78971105,NaN,"[{'name': 'Python'}, {'name': 'Статистический ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Мы сохранили в себе важный плюс от стартапа...,Product manager (machine learning) Менеджер пр...,NaN
26,Product manager (machine learning),Менеджер продукта,LC Group,Екатеринбург,NaN,NaN,NaN,Профильное математическое образование. Не мене...,Сбор и обработка данных с помощью SQL и Python...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/78971108?host=hh.ru,Полная занятость,78971108,NaN,"[{'name': 'Python'}, {'name': 'Статистический ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Мы сохранили в себе важный плюс от стартапа...,Product manager (machine learning) Менеджер пр...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129,Младший аналитик Центра прикладного анализа бо...,Аналитик,Национальный исследовательский Томский государ...,Томск,NaN,NaN,NaN,Высшее/неоконченное профессиональное образован...,Проведение аналитической и научно-исследовател...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81270894?host=hh.ru,Полная занятость,81270894,NaN,"[{'name': 'Статистический анализ'}, {'name': '...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<strong>Обязанности:</strong> <ul> <li> <p>Про...,Младший аналитик Центра прикладного анализа бо...,NaN
2134,Научный сотрудник (постдок) в НУЛ методов анал...,"Научный специалист, исследователь",Национальный исследовательский университет Выс...,Москва,NaN,NaN,NaN,"Учёную степень по науке о данных, статистике и...",Проводить исследования в одной или нескольких ...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81409175?host=hh.ru,Полная занятость,81409175,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}","<p>Мы, <strong>Высшая школа экономики</strong>...",Научный сотрудник (постдок) в НУЛ методов анал...,NaN
2135,Начальник отдела по обработке и визуализации п...,Архитектор,ГКУ ТО Центр информационных технологий Тюменск...,Тюмень,70000.0,NaN,RUR,Высшее образование в области IT. Опыт работы в...,Руководство отделом. Декомпозиция поступающих ...,От 

In [41]:
df = df[(df.profession.isnull()==False)]

In [42]:
df.shape

(1392, 22)

In [43]:
df

,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,...,url,employment.name,id,salary.gross,key_skills,languages,schedule,description,common_role,profession
0,Machine Learning Engineer,Дата-сайентист,ДОМ.РФ,Москва,NaN,NaN,NaN,Решать амбициозные задачи из области NLP. Увер...,Принимать технические и архитектурные решения ...,Нет опыта,...,https://api.hh.ru/vacancies/81645171?host=hh.ru,Полная занятость,81645171,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>ЧЕМ НЕОБХОДИМО ЗАНИМАТЬСЯ:</strong>...,Machine Learning Engineer Дата-сайентист,Data Scientist/ML developer
2,Junior Data scientist/Machine learning,Аналитик,СБЕР,Москва,NaN,NaN,NaN,Обучение классических моделей машинного обучен...,Анализ и использование различных источников да...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81456638?host=hh.ru,Полная занятость,81456638,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ищем Data scientist в Блок Сервисы. Это Бло...,Junior Data scientist/Machine learning Аналитик,Data Scientist/ML developer
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,250000.0,300000.0,RUR,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81253945?host=hh.ru,Частичная занятость,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Задачи:</strong></p> <ul> <li>Приду...,Machine Learning Engineer (ML) Другое,Data Scientist/ML developer
5,Machine Learning Engineer,"Программист, разработчик",НеоБИТ,Санкт-Петербург,NaN,NaN,NaN,"Знания Мат. аппарата (мат. статистики, теории ...",Исследование и реализация методов предобработк...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/80948681?host=hh.ru,Полная занятость,80948681,NaN,"[{'name': 'Machine Learning'}, {'name': 'Compu...",[],"{'id': 'fullDay', 'name': 'Полный день'}","<p>Приглашаем в команду специалиста по ML,кото...","Machine Learning Engineer Программист, разрабо...",Data Scientist/ML developer
6,Data scientist / Machine learning engineer (ко...,"Программист, разработчик",VK,Санкт-Петербург,NaN,NaN,NaN,Математически формулировать бизнес-задачи. С п...,Использовать огромное количество разных данных...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81833434?host=hh.ru,Полная занятость,81833434,NaN,"[{'name': 'Python'}, {'name': 'Machine Learnin...",[],"{'id': 'flexible', 'name': 'Гибкий график'}",<p>Мы применяем алгоритмы машинного обучения к...,Data scientist / Machine learning engineer (ко...,Data Scientist/ML developer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,Middle Дата Аналитик в продукт Личные Финансы ...,Аналитик,МТС,Москва,NaN,NaN,NaN,Знание статистических методов и умение проводи...,"Проводить A/B тесты (дизайн, анализ, интерпрет...",От 3 до 6 лет,...,https://api.hh.ru/vacancies/81498254?host=hh.ru,Полная занятость,81498254,NaN,"[{'name': 'A/B тесты'}, {'name': 'SQL'}, {'nam...",[],"{'id': 'fullDay', 'name': 'Полный день'}","<p><strong>Big Data МТС</strong> – место, где ...",Middle Дата Аналитик в продукт Личные Финансы ...,Data analyst
2140,Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,NaN,Опыт написания технических заданий на разработ...,Участие в создании бизнес-метрик продукта. Пло...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/81088795?host=hh.ru,Полная занятость,81088795,NaN,[],[],"{'id': 'remote', 'name': 'Удаленная работа'}","<p><strong>Итак, тебе предстоит:</strong></p> ...","Аналитик больших данных BI-аналитик, аналитик ...",Data analyst
2141,Старший Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,NaN,Опыт работы аналитиком от 2 лет. Развитые комм...,"Сбор, анализ и разработка (уточнение и детализ...",От 1 года до 3 лет,...,https://api.hh.ru/vacancies/78357091?host=hh.ru,Полная занятость,7835709

### salary

Fill in missing values when there is salary.from but no salary.to and vice versa.

Заполним отсутствующие значения, когда есть salary.from но нет salary.to и наоборот.

In [44]:
df.rename(columns = {'salary.from':'salary_from', 'salary.to':'salary_to'}, inplace = True )

In [45]:
# Calculate the median share between salaries for
# vacancies where there is both an upper and lower range.
#
# Посчитаем по вакансиям, где есть и верхний и нижний
# диапазон, медианную разницу между зарплатами.
df_dif_salary = df[(df.salary_from.isnull()==False)&(df.salary_to.isnull()==False)]

In [46]:
df_dif_salary['dif'] = (df_dif_salary.salary_to - df_dif_salary.salary_from) / df_dif_salary.salary_from

In [47]:
df_dif_salary['dif'].median()

0.42857142857142855

In [48]:
df_dif_salary

,name,role,employer.name,area.name,salary_from,salary_to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,...,employment.name,id,salary.gross,key_skills,languages,schedule,description,common_role,profession,dif
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,250000.0,300000.0,RUR,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,...,Частичная занятость,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Задачи:</strong></p> <ul> <li>Приду...,Machine Learning Engineer (ML) Другое,Data Scientist/ML developer,0.200000
35,Senior Machine Learning Engineer,"Программист, разработчик",MetaQuotes Software Corp.,Кипр,5500.0,7000.0,EUR,Опыт в проектах по моделированию в бизнес-орие...,Разработка систем поведенческого анализа и кла...,Более 6 лет,...,Полная занятость,79910163,True,"[{'name': 'Machine Learning'}, {'name': 'C++'}...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><em><strong>Внимание</strong></em>. <em>Вак...,"Senior Machine Learning Engineer Программист, ...",Data Scientist/ML developer,0.272727
43,ML-Engineer,"Программист, разработчик",Нейросети,Москва,250000.0,350000.0,RUR,Приветствую. Требуемый опыт: Middle+/Senior. -...,Создание диалоговой системы с возможностью обр...,От 3 до 6 лет,...,Полная занятость,81916611,False,"[{'name': 'Нейронные сети'}, {'name': 'ML'}, {...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>ML</strong><strong>-</strong><stron...,"ML-Engineer Программист, разработчик",Data Scientist/ML developer,0.400000
51,ML engineer,Дата-сайентист,EGISTIC,Астана,600000.0,1000000.0,KZT,"Знание Python (Pandas, NumPy, Seaborn, Scikit ...",Сбор и разметках данных. Написание систем расп...,От 3 до 6 лет,...,Полная занятость,81499736,True,"[{'name': 'Data Analysis'}, {'name': 'Математи...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Обязанности:</strong></p> <p>сбор и...,ML engineer Дата-сайентист,Data Scientist/ML developer,0.666667
62,Senior ML Engineer,"Программист, разработчик",Improvado KZ,Москва,5500.0,8000.0,USD,Опыт и понимание моделей LLM. Большим плюсом б...,Настройка тестирования продуктов ML. Процесс н...,От 1 года до 3 лет,...,Полная занятость,81915800,True,"[{'name': 'Python'}, {'name': 'Английский язык...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p><strong>О компании:</strong><br />Основной ...,"Senior ML Engineer Программист, разработчик",Data Scientist/ML developer,0.454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672,"Аналитик данных (удаленно, junior/middle)","BI-аналитик, аналитик данных",Outlines Technologies,Москва,100000.0,130000.0,RUR,Опыт на позиции <highlighttext>аналитика</high...,Сбор <highlighttext>аналитики</highlighttext> ...,От 3 до 6 лет,...,Полная занятость,81348742,False,"[{'name': 'Hadoop'}, {'name': 'Data Analysis'}...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Outlines Tech – это аккредитованная IT-комп...,"Аналитик данных (удаленно, junior/middle) BI-а...",Data analyst,0.300000
1678,Аналитик данных,"BI-аналитик, аналитик данных",Unitiki,Москва,180000.0,180000.0,RUR,Ищем специалиста по анализу <highlighttext>дан...,Мы продаем автобусные билеты и разрабатываем п...,От 3 до 6 лет,...,Полная занятость,81744993,False,"[{'name': 'SQL'}, {'name': 'Анализ данных'}, {...",[],"{'id': 'remote', 'name': 'Удаленная работа'}","<p>IT-компания Unitiki, основанная в 2015 г., ...","Аналитик данных BI-аналитик, аналитик данных",Data analyst,0.000000
1792,Аналитик данных,Менеджер по закупкам,Васильева Наталия Сергеевна,Санкт-Петербург,69000.0,92000.0,RUR,Понимание предметной области. Знание методов а...,Интеграция и внедрение единой базы <highlightt...,От 1 года до 3 лет,...,Полная занятость,81366611,True,"[{'name': 'Работа с документами'}, {'name': 'А...","[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'fullDay', 'name': 'Полный 

In [49]:
df[(df.salary_from.isnull()==False) &
   (df.salary_to.isnull()==True)]

,name,role,employer.name,area.name,salary_from,salary_to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,...,url,employment.name,id,salary.gross,key_skills,languages,schedule,description,common_role,profession
10,Team Lead Machine Learning (Computer Vision),Руководитель группы разработки,EKSLi,Москва,250000.0,NaN,RUR,Опыт работы в роли Team Lead от 1 года. Опыт р...,Проектирование систем машинного зрения для сор...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/82040145?host=hh.ru,Полная занятость,82040145,False,[],"[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Лидер команды разработки системы </...,Team Lead Machine Learning (Computer Vision) Р...,Data Scientist/ML developer
65,ML Engineer,Дата-сайентист,Университет ИТМО,Санкт-Петербург,150000.0,NaN,RUR,"Продуктовый опыт в 2 областях ML, приоритет (C...",Проводить решение через полный цикл ML разрабо...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/78393436?host=hh.ru,Полная занятость,78393436,True,"[{'name': 'Python'}, {'name': 'Docker'}, {'nam...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Чем предстоит заниматься:</strong><...,ML Engineer Дата-сайентист,Data Scientist/ML developer
69,ML-инженер (Банковский скоринг),"BI-аналитик, аналитик данных",Займиго МФК,Москва,150000.0,NaN,RUR,Серьезную математическую подготовку. Опыт прим...,Построение рисковых скоринговых моделей. Монит...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/80884767?host=hh.ru,Полная занятость,80884767,False,"[{'name': 'Анализ рисков'}, {'name': 'Аналитич...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Zaymigo – </strong>это современная ...,"ML-инженер (Банковский скоринг) BI-аналитик, а...",Data Scientist/ML developer
271,Data Scientist / ML-специалист,Дата-сайентист,КАМИ,Москва,150000.0,NaN,RUR,Опыт использования и внедрения алгоритмов маши...,Анализ имеющихся данных. Подготовка данных для...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81316294?host=hh.ru,Полная занятость,81316294,True,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Мы ищем сотрудника на проект по созданию мо...,Data Scientist / ML-специалист Дата-сайентист,Data Scientist/ML developer
357,Специалист по компьютерному зрению и машинному...,"Программист, разработчик",ИТС,Тюмень,60000.0,NaN,RUR,"Знания Python, фреймворков TensorFlow, PyTorch...",Работа с полным циклом задач машинного обучени...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/80925176?host=hh.ru,Полная занятость,80925176,False,"[{'name': 'PyTorch'}, {'name': 'CUDA'}, {'name...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><em>Приветствую соискатель! Мы команда моло...,Специалист по компьютерному зрению и машинному...,Data Scientist/ML developer
444,Senior Project Manager (MLOps),Руководитель проектов,Rubbles,Москва,200000.0,NaN,RUR,Опыт работы от 3 лет в управлении технологичес...,Управление проектами: Планирование работ по пр...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/81924271?host=hh.ru,Полная занятость,81924271,True,"[{'name': 'Project management'}, {'name': 'Про...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Мы в <strong>Rubbles </strong>занимаемся со...,Senior Project Manager (MLOps) Руководитель пр...,MLOps
478,Junior Data scientist (Аналитик),Аналитик,Займиго МФК,Москва,60000.0,NaN,RUR,Серьезную математическую подготовку. Опыт прим...,Построение рисковых скоринговых моделей. Монит...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81862820?host=hh.ru,Полная занятость,81862820,False,"[{'name': 'Python'}, {'name': 'Статистический ...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Zaymigo – </strong>это современная ...,Junior Data scientist (Аналитик) Аналитик,Data Scientist/ML developer
481,Junior Data scientist (Аналитик),Аналитик,Займиго МФК,Нижний Новгород,60000.0,NaN,RUR,Серьезную математическую подготовку. Опыт прим...,Построение рисковых скоринговых моделей. Монит...,Нет опыта,...,https://api.hh.r

In [50]:
# Fill in the salary_to values when there is a salary_from 
# but no salary_to, using the obtained median value of the
# difference in the salary fork.
#
# Заполним значения salary_to когда есть salary_from но нет 
# salary_to, используя полученное медианное значение разницы
# в зарплатной вилке.
df.loc[(df.salary_from.isnull()==False) & 
       (df.salary_to.isnull()==True), 'salary_to'] = (df.salary_from[(df.salary_from.isnull()==False) & 
                                                     (df.salary_to.isnull()==True)])*0.43 + (df.salary_from[(df.salary_from.isnull()==False) &
                                                     (df.salary_to.isnull()==True)])

In [51]:
# Fill in the salary_from values when there is no salary_from
# but there is a salary_to, using the obtained median value
# of the difference in the salary fork.
#
# Заполним значения salary_from когда нет salary_from но есть
# salary_to, используя полученное медианное значение разницы 
# в зарплатной вилке.
df.loc[(df.salary_from.isnull()==True) & 
       (df.salary_to.isnull()==False), 'salary_from'] = (df.salary_to[(df.salary_from.isnull()==True) & 
                                                        (df.salary_to.isnull()==False)])/1.43

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1392 entries, 0 to 2238
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    1392 non-null   object 
 1   role                    1392 non-null   object 
 2   employer.name           1392 non-null   object 
 3   area.name               1392 non-null   object 
 4   salary_from             188 non-null    float64
 5   salary_to               188 non-null    float64
 6   salary.currency         188 non-null    object 
 7   snippet.requirement     1384 non-null   object 
 8   snippet.responsibility  1380 non-null   object 
 9   experience.name         1392 non-null   object 
 10  accept_temporary        1392 non-null   bool   
 11  published_at            1392 non-null   object 
 12  url                     1392 non-null   object 
 13  employment.name         1392 non-null   object 
 14  id                      1392 non-null   

In [53]:
df[(df.salary_from.isnull()==False)]

,name,role,employer.name,area.name,salary_from,salary_to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,...,url,employment.name,id,salary.gross,key_skills,languages,schedule,description,common_role,profession
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,250000.000000,300000.0,RUR,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81253945?host=hh.ru,Частичная занятость,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Задачи:</strong></p> <ul> <li>Приду...,Machine Learning Engineer (ML) Другое,Data Scientist/ML developer
10,Team Lead Machine Learning (Computer Vision),Руководитель группы разработки,EKSLi,Москва,250000.000000,357500.0,RUR,Опыт работы в роли Team Lead от 1 года. Опыт р...,Проектирование систем машинного зрения для сор...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/82040145?host=hh.ru,Полная занятость,82040145,False,[],"[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Лидер команды разработки системы </...,Team Lead Machine Learning (Computer Vision) Р...,Data Scientist/ML developer
35,Senior Machine Learning Engineer,"Программист, разработчик",MetaQuotes Software Corp.,Кипр,5500.000000,7000.0,EUR,Опыт в проектах по моделированию в бизнес-орие...,Разработка систем поведенческого анализа и кла...,Более 6 лет,...,https://api.hh.ru/vacancies/79910163?host=hh.ru,Полная занятость,79910163,True,"[{'name': 'Machine Learning'}, {'name': 'C++'}...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><em><strong>Внимание</strong></em>. <em>Вак...,"Senior Machine Learning Engineer Программист, ...",Data Scientist/ML developer
43,ML-Engineer,"Программист, разработчик",Нейросети,Москва,250000.000000,350000.0,RUR,Приветствую. Требуемый опыт: Middle+/Senior. -...,Создание диалоговой системы с возможностью обр...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/81916611?host=hh.ru,Полная занятость,81916611,False,"[{'name': 'Нейронные сети'}, {'name': 'ML'}, {...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>ML</strong><strong>-</strong><stron...,"ML-Engineer Программист, разработчик",Data Scientist/ML developer
45,ML Engineer,"Программист, разработчик",СБЕР,Санкт-Петербург,188811.188811,270000.0,RUR,Имеете опыт разработки моделей машинного обуче...,"Изучение научных статей и SOTA подходов, генер...",От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81925449?host=hh.ru,Полная занятость,81925449,True,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Информация о проекте</strong></p> <...,"ML Engineer Программист, разработчик",Data Scientist/ML developer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1896,Ведущий аналитик данных,Аналитик,ГКУ города Москвы Центр занятости населения го...,Москва,150000.000000,214500.0,RUR,...<highlighttext>данных</highlighttext>. Увер...,Развитие алгоритмов связывания <highlighttext>...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81085748?host=hh.ru,Полная занятость,81085748,True,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Один из ключевых партнеров Центра занятости...,Ведущий аналитик данных Аналитик,Data analyst
1901,Аналитик данных (Главный специалист Департамен...,"BI-аналитик, аналитик данных",Объединенная Авиастроительная Корпорация,Москва,90909.090909,130000.0,RUR,Высшее техническое или экономическое образован...,Сбор и анализ <highlighttext>данных</highlight...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/82050957?host=hh.ru,Полная занятость,82050957,True,"[{'name': 'MS Excel (VBA, Power Query), Python'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Обязанности:</strong></p> <ul> <li>...,Аналитик данных (Главный специалист Департамен...,Data analyst
1917,Аналитик данных (отдел регулирования продаж),Аналитик,DNS Головной офис,Владивосток,

The salary, where it is, must be brought to one currency. We will translate into dollars, the exchange rate on 06/21/23:

Зарплату, где она есть, нужно привести к одной валюте. Будем переводит в доллары, курс на 21.06.23:

- 1$ = 84,23 RUB

- 1$ = 0,91484 EUR

- 1$ = 447,15 KZT

In [54]:
df['salary.currency'].value_counts()

RUR    155
USD     16
EUR     12
KZT      5
Name: salary.currency, dtype: int64

In [55]:
df['salary_from'][df['salary.currency']=='RUR'] = df['salary_from'][df['salary.currency']=='RUR']/(84.23)

In [56]:
df['salary_to'][df['salary.currency']=='RUR'] = df['salary_to'][df['salary.currency']=='RUR']/(84.23)

In [57]:
df['salary_from'][df['salary.currency']=='EUR'] = df['salary_from'][df['salary.currency']=='EUR']/(0.91484)

In [58]:
df['salary_to'][df['salary.currency']=='EUR'] = df['salary_to'][df['salary.currency']=='EUR']/(0.91484)

In [59]:
df['salary_from'][df['salary.currency']=='KZT'] = df['salary_from'][df['salary.currency']=='KZT']/(447.15)

In [60]:
df['salary_to'][df['salary.currency']=='KZT'] = df['salary_to'][df['salary.currency']=='KZT']/(447.15)

In [61]:
df[df['salary.currency']=='RUR']

,name,role,employer.name,area.name,salary_from,salary_to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,...,url,employment.name,id,salary.gross,key_skills,languages,schedule,description,common_role,profession
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,2968.063635,3561.676362,RUR,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81253945?host=hh.ru,Частичная занятость,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Задачи:</strong></p> <ul> <li>Приду...,Machine Learning Engineer (ML) Другое,Data Scientist/ML developer
10,Team Lead Machine Learning (Computer Vision),Руководитель группы разработки,EKSLi,Москва,2968.063635,4244.330998,RUR,Опыт работы в роли Team Lead от 1 года. Опыт р...,Проектирование систем машинного зрения для сор...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/82040145?host=hh.ru,Полная занятость,82040145,False,[],"[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Лидер команды разработки системы </...,Team Lead Machine Learning (Computer Vision) Р...,Data Scientist/ML developer
43,ML-Engineer,"Программист, разработчик",Нейросети,Москва,2968.063635,4155.289089,RUR,Приветствую. Требуемый опыт: Middle+/Senior. -...,Создание диалоговой системы с возможностью обр...,От 3 до 6 лет,...,https://api.hh.ru/vacancies/81916611?host=hh.ru,Полная занятость,81916611,False,"[{'name': 'Нейронные сети'}, {'name': 'ML'}, {...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>ML</strong><strong>-</strong><stron...,"ML-Engineer Программист, разработчик",Data Scientist/ML developer
45,ML Engineer,"Программист, разработчик",СБЕР,Санкт-Петербург,2241.614494,3205.508726,RUR,Имеете опыт разработки моделей машинного обуче...,"Изучение научных статей и SOTA подходов, генер...",От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81925449?host=hh.ru,Полная занятость,81925449,True,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Информация о проекте</strong></p> <...,"ML Engineer Программист, разработчик",Data Scientist/ML developer
50,ML Engineer,"Программист, разработчик",Eqwile,Москва,2490.682771,3561.676362,RUR,Уверенные знания Python. Опыт работы с библиот...,Сбор и предобработка данных для обучения. Разр...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81758883?host=hh.ru,Полная занятость,81758883,True,"[{'name': 'Python'}, {'name': 'Clickhouse'}, {...",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Более 13 лет мы создаем ИТ – решени...,"ML Engineer Программист, разработчик",Data Scientist/ML developer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1896,Ведущий аналитик данных,Аналитик,ГКУ города Москвы Центр занятости населения го...,Москва,1780.838181,2546.598599,RUR,...<highlighttext>данных</highlighttext>. Увер...,Развитие алгоритмов связывания <highlighttext>...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/81085748?host=hh.ru,Полная занятость,81085748,True,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Один из ключевых партнеров Центра занятости...,Ведущий аналитик данных Аналитик,Data analyst
1901,Аналитик данных (Главный специалист Департамен...,"BI-аналитик, аналитик данных",Объединенная Авиастроительная Корпорация,Москва,1079.295867,1543.393090,RUR,Высшее техническое или экономическое образован...,Сбор и анализ <highlighttext>данных</highlight...,От 1 года до 3 лет,...,https://api.hh.ru/vacancies/82050957?host=hh.ru,Полная занятость,82050957,True,"[{'name': 'MS Excel (VBA, Power Query), Python'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Обязанности:</strong></p> <ul> <li>...,Аналитик данных (Главный специалист Департамен...,Data analyst
1917,Аналитик данных (отдел регулирования продаж),Аналитик,DNS Головной офис,Владивосток,712.335272,949.780363,RUR,Желани

In [62]:
# The salary.currency column is no longer needed, remove it.
#
# Столбец salary.currency больше не нужен, удалим его.
df = df.drop(['salary.currency'], axis=1)

In [63]:
df['salary_from'][(df.salary_from.isnull()==False)] = df['salary_from'][(df.salary_from.isnull()==False)].astype('int64')
df['salary_to'][(df.salary_to.isnull()==False)] = df['salary_to'][(df.salary_to.isnull()==False)].astype('int64')

### area.name

Using this column we can add a new attribute **country**

По этому столбцу мы можем добавить новый признак **country**

In [64]:
df['area.name'].unique()

array(['Москва', 'Санкт-Петербург', 'Пермь', 'Минск', 'Тбилиси', 'Сербия',
       'Краснодар', 'Кипр', 'Армения', 'Астана', 'Казань', 'Екатеринбург',
       'Нижний Новгород', 'Уфа', 'Алматы', 'Рязань', 'Новосибирск',
       'Ульяновск', 'Томск', 'Ижевск', 'Воронеж', 'Иннополис',
       'Чебоксары', 'Челябинск', 'Ярославль', 'Омск', 'Ростов-на-Дону',
       'Саратов', 'Самара', 'Сочи', 'Уральск', 'Тюмень', 'Йошкар-Ола',
       'Владивосток', 'Ташкент', 'Кострома', 'Южно-Сахалинск',
       'Красноярск', 'Хабаровск', 'Таганрог', 'Иркутск', 'Смоленск',
       'Московский', 'США', 'Сириус', 'Владимир', 'Липецк',
       'Благовещенск (Амурская область)', 'Турция', 'Тула', 'Атырау',
       'Баку', 'Аксай (Казахстан)', 'Химки', 'Симферополь', 'Бишкек',
       'Тверь', 'Тольятти', 'Калининград', 'Фрязино'], dtype=object)

In [65]:
RF_city = ['Москва', 'Санкт-Петербург', 'Новосибирск', 'Пермь', 
          'Екатеринбург', 'Краснодар', 'Казань', 'Нижний Новгород', 'Уфа', 
          'Рязань', 'Ульяновск', 'Томск', 'Ижевск', 'Воронеж', 'Иннополис',
          'Чебоксары', 'Челябинск', 'Ярославль', 'Омск', 'Ростов-на-Дону',
          'Саратов', 'Самара', 'Сочи',  'Тюмень', 'Йошкар-Ола', 'Владивосток',
          'Кострома', 'Южно-Сахалинск', 'Красноярск', 'Хабаровск', 'Таганрог',
          'Иркутск', 'Смоленск', 'Московский',  'Сириус', 'Владимир', 'Липецк',
          'Благовещенск (Амурская область)','Тула', 'Химки', 'Вологда', 'Симферополь',
          'Тверь', 'Тольятти', 'Ахтубинск', 'Калининград', 'Фрязино']
KZ_city = ['Астана', 'Алматы', 'Уральск', 'Атырау', 'Аксай (Казахстан)']
USA_city = ['США']
KG_city = ['Бишкек']
Armenia_city = ['Армения']
Georgia_city = ['Тбилиси']
Belarus_city =['Минск']
Serbia_city = ['Сербия']
Cyprus_city = ['Кипр']
Uzbekistan_city = ['Ташкент']
Turkey_city = ['Турция']
Azerbaijan_city = ['Баку']

In [66]:
df.loc[(df['area.name'].isin(RF_city) == True), 'country'] = 'Russia'
df.loc[(df['area.name'].isin(KZ_city) == True), 'country'] = 'Kazakhstan'
df.loc[(df['area.name'].isin(USA_city) == True), 'country'] = 'USA'
df.loc[(df['area.name'].isin(KG_city) == True), 'country'] = 'Kyrgyzstan'
df.loc[(df['area.name'].isin(Armenia_city) == True), 'country'] = 'Armenia'
df.loc[(df['area.name'].isin(Georgia_city) == True), 'country'] = 'Georgia'
df.loc[(df['area.name'].isin(Belarus_city) == True), 'country'] = 'Belarus'
df.loc[(df['area.name'].isin(Serbia_city) == True), 'country'] = 'Serbia'
df.loc[(df['area.name'].isin(Cyprus_city) == True), 'country'] = 'Cyprus'
df.loc[(df['area.name'].isin(Uzbekistan_city) == True), 'country'] = 'Uzbekistan'
df.loc[(df['area.name'].isin(Turkey_city) == True), 'country'] = 'Turkey'
df.loc[(df['area.name'].isin(Azerbaijan_city) == True), 'country'] = 'Azerbaijan'

In [67]:
df.to_csv('reached_data2.csv', index=False)

### languages

In [68]:
df

,name,role,employer.name,area.name,salary_from,salary_to,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,...,employment.name,id,salary.gross,key_skills,languages,schedule,description,common_role,profession,country
0,Machine Learning Engineer,Дата-сайентист,ДОМ.РФ,Москва,NaN,NaN,Решать амбициозные задачи из области NLP. Увер...,Принимать технические и архитектурные решения ...,Нет опыта,False,...,Полная занятость,81645171,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>ЧЕМ НЕОБХОДИМО ЗАНИМАТЬСЯ:</strong>...,Machine Learning Engineer Дата-сайентист,Data Scientist/ML developer,Russia
2,Junior Data scientist/Machine learning,Аналитик,СБЕР,Москва,NaN,NaN,Обучение классических моделей машинного обучен...,Анализ и использование различных источников да...,От 1 года до 3 лет,False,...,Полная занятость,81456638,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ищем Data scientist в Блок Сервисы. Это Бло...,Junior Data scientist/Machine learning Аналитик,Data Scientist/ML developer,Russia
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,2968.0,3561.0,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,True,...,Частичная занятость,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Задачи:</strong></p> <ul> <li>Приду...,Machine Learning Engineer (ML) Другое,Data Scientist/ML developer,Russia
5,Machine Learning Engineer,"Программист, разработчик",НеоБИТ,Санкт-Петербург,NaN,NaN,"Знания Мат. аппарата (мат. статистики, теории ...",Исследование и реализация методов предобработк...,От 1 года до 3 лет,False,...,Полная занятость,80948681,NaN,"[{'name': 'Machine Learning'}, {'name': 'Compu...",[],"{'id': 'fullDay', 'name': 'Полный день'}","<p>Приглашаем в команду специалиста по ML,кото...","Machine Learning Engineer Программист, разрабо...",Data Scientist/ML developer,Russia
6,Data scientist / Machine learning engineer (ко...,"Программист, разработчик",VK,Санкт-Петербург,NaN,NaN,Математически формулировать бизнес-задачи. С п...,Использовать огромное количество разных данных...,От 1 года до 3 лет,False,...,Полная занятость,81833434,NaN,"[{'name': 'Python'}, {'name': 'Machine Learnin...",[],"{'id': 'flexible', 'name': 'Гибкий график'}",<p>Мы применяем алгоритмы машинного обучения к...,Data scientist / Machine learning engineer (ко...,Data Scientist/ML developer,Russia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,Middle Дата Аналитик в продукт Личные Финансы ...,Аналитик,МТС,Москва,NaN,NaN,Знание статистических методов и умение проводи...,"Проводить A/B тесты (дизайн, анализ, интерпрет...",От 3 до 6 лет,False,...,Полная занятость,81498254,NaN,"[{'name': 'A/B тесты'}, {'name': 'SQL'}, {'nam...",[],"{'id': 'fullDay', 'name': 'Полный день'}","<p><strong>Big Data МТС</strong> – место, где ...",Middle Дата Аналитик в продукт Личные Финансы ...,Data analyst,Russia
2140,Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,Опыт написания технических заданий на разработ...,Участие в создании бизнес-метрик продукта. Пло...,От 3 до 6 лет,False,...,Полная занятость,81088795,NaN,[],[],"{'id': 'remote', 'name': 'Удаленная работа'}","<p><strong>Итак, тебе предстоит:</strong></p> ...","Аналитик больших данных BI-аналитик, аналитик ...",Data analyst,Russia
2141,Старший Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,Опыт работы аналитиком от 2 лет. Развитые комм...,"Сбор, анализ и разработка (уточнение и детализ...",От 1 года до 3 лет,False,...,Полная занятость,78357091,NaN,"[{'name': 'Python'}, {'name': 'Spark'}, {'name...",[],"{'id': 'fullDay', 'name': 'Полный день'}","<p>Создание и развитие цифровых инструментов, ...","Старший Аналитик больших данных BI-аналитик, а...",Data analyst,Russia
2142,Эксперт по исследованиям больших данных,"BI-аналитик, аналитик данных",UNITEL LLC,Ташк

In [69]:
# replace gaps with an empty list.
#
# замена пропусков на пустой список.
def empty_assign(s):
    idx = s.isna()
    s.loc[idx] = [[]] * idx.sum()

In [70]:
empty_assign(df['languages'])

In [71]:
def extract_language(text):
    #print(text)
    #text = text.to_json(orient='records', force_ascii=False)
    ##text=loads(text)
    #text=text[0]
    let_list = []
    for i in range(len(text)):
        if text[i] not in "[{',:}]":
            let_list.append(text[i])
    let_list = let_list[3:]
    let_list = "".join(let_list)
    let_list = let_list.replace('name', '')
    let_list = let_list.replace('level id', '')
    let_list = let_list.split('  ')
    if len(let_list)>=3:
        let_list.pop(1)
        let_list.pop(2)
        return let_list[0]

In [72]:
def extract_language_level(text):
    let_list = []
    for i in range(len(text)):
        if text[i] not in "[{',:}]":
            let_list.append(text[i])
    let_list = let_list[3:]
    let_list = "".join(let_list)
    let_list = let_list.replace('name', '')
    let_list = let_list.replace('level id', '')
    let_list = let_list.split('  ')
    if len(let_list)>=3:
        let_list.pop(1)
        let_list.pop(2)
        return let_list[1]

In [73]:
df['language'] = df['languages'].apply(extract_language)

In [74]:
df['language_level'] = df['languages'].apply(extract_language_level)

In [75]:
df[df.language.notnull()]

,name,role,employer.name,area.name,salary_from,salary_to,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,...,salary.gross,key_skills,languages,schedule,description,common_role,profession,country,language,language_level
10,Team Lead Machine Learning (Computer Vision),Руководитель группы разработки,EKSLi,Москва,2968.0,4244.0,Опыт работы в роли Team Lead от 1 года. Опыт р...,Проектирование систем машинного зрения для сор...,От 1 года до 3 лет,False,...,False,[],"[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Лидер команды разработки системы </...,Team Lead Machine Learning (Computer Vision) Р...,Data Scientist/ML developer,Russia,eng,b2
60,ML-инженер,"Программист, разработчик","Программный Продукт, ИТ-компания",Москва,NaN,NaN,Профессиональное владение Python-стеком (NumPy...,Работа с большими объемами разнотипных данных....,От 3 до 6 лет,False,...,NaN,[],"[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'fullDay', 'name': 'Полный день'}",<strong>Обязанности:</strong> <ul> <li>работа ...,"ML-инженер Программист, разработчик",Data Scientist/ML developer,Russia,eng,a1
138,"Senior AI/ML engineer, to Hungary","Программист, разработчик",SCHNEIDER GROUP,Москва,NaN,NaN,Bachelors/Master’s degree in engineering or co...,Develop AI/ML solutions to enhance in-vehicle ...,Более 6 лет,False,...,NaN,[],"[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'fullDay', 'name': 'Полный день'}",<p>Senior AI/ML engineer Engineer (R&amp;D dep...,"Senior AI/ML engineer, to Hungary Программист,...",Data Scientist/ML developer,Russia,eng,b2
148,Senior ML Engineer,"Научный специалист, исследователь",Пану Ольга Германовна,Москва,NaN,NaN,"1. In-depth knowledge of: Machine Learning, De...",The company develops a system capable of seman...,От 3 до 6 лет,False,...,NaN,"[{'name': 'Machine Learning'}, {'name': 'Deep ...","[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'remote', 'name': 'Удаленная работа'}",<p><em>The company develops a system capable o...,"Senior ML Engineer Научный специалист, исследо...",Data Scientist/ML developer,Russia,eng,c1
269,ML-специалист (Junior),"Программист, разработчик",ГравиЛинк,Санкт-Петербург,1000.0,2000.0,Имели базовые знания математики и информационн...,Исследованием и реализацией методов предобрабо...,Нет опыта,False,...,True,"[{'name': 'Английский язык - А2'}, {'name': 'И...","[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'flexible', 'name': 'Гибкий график'}","<p>Большое количество людей пропадает, теряетс...","ML-специалист (Junior) Программист, разработчик",Data Scientist/ML developer,Russia,eng,a2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,Аналитик данных (контроль качества данных),Аналитик,Центральный банк Российской Федерации,Москва,NaN,NaN,Высшее техническое/математическое образование....,...<highlighttext>данных</highlighttext>. Разр...,От 3 до 6 лет,False,...,NaN,"[{'name': 'SQL'}, {'name': 'ORACLE'}, {'name':...","[{'id': 'eng', 'name': 'Английский', 'level': ...","{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Обязанности:</strong></p> <ul> <li>...,Аналитик данных (контроль качества данных) Ана...,Data analyst,Russia,eng,b1
1955,Старший Аналитик Данных (Ташкент),Аналитик,«UZUM TECHNOLOGIES».,Москва,NaN,NaN,"SQL - хороший уровень (CTE, оконные функции). ...",Python для анализа <highlighttext>данных</high...,От 3 до 6 лет,False,...,NaN,"[{'name': 'SQL'}, {'name': 'Python'}]","[{'id': 'rus', 'name': 'Русский', 'level': {'i...","{'id': 'fullDay', 'name': 'Полный день'}",<p><em><strong>Кто мы</strong></em></p> <p><em...,Старший Аналитик Данных (Ташкент) Аналитик,Data analyst,Russia,rus,c2
2022,Системный аналитик/Аналитик данных,Аналитик,СЕУСЛАБ,Пермь,NaN,NaN,Умение находить общий язык и договариваться в ...,Разработка моделей предметной области (онтолог...,От 3 до 6 лет,False,...,NaN,"[{'name': 'UML'}, {'name': 'Управление требова...","[{'i

In [76]:
# languages column is no longer needed, remove it.
#
# столбец languages больше не нужен, удалим его.
df = df.drop(['languages'], axis=1)

### schedule	

In [77]:
def extract_schedule(text):
    let_list = []
    for i in range(len(text)):
        if text[i] not in "[{',:}]":
            let_list.append(text[i])
    let_list = let_list[3:]   
    let_list = "".join(let_list)
    let_list = let_list.split(' ')
    return let_list[0]

In [78]:
empty_assign(df['schedule'])

In [79]:
df['schedule'] = df['schedule'].apply(extract_schedule)

In [80]:
from json import loads
text = df.schedule[df.id==81021174	]
text = text.to_json(orient='records', force_ascii=False)
text=loads(text)
text=text[0]
let_list = []
for i in range(len(text)):
    if text[i] not in "[{',:}]":
        let_list.append(text[i])    
let_list = let_list[3:]   
let_list = "".join(let_list)
let_list = let_list.split(' ')

In [81]:
df

,name,role,employer.name,area.name,salary_from,salary_to,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,...,id,salary.gross,key_skills,schedule,description,common_role,profession,country,language,language_level
0,Machine Learning Engineer,Дата-сайентист,ДОМ.РФ,Москва,NaN,NaN,Решать амбициозные задачи из области NLP. Увер...,Принимать технические и архитектурные решения ...,Нет опыта,False,...,81645171,NaN,[],fullDay,<p><strong>ЧЕМ НЕОБХОДИМО ЗАНИМАТЬСЯ:</strong>...,Machine Learning Engineer Дата-сайентист,Data Scientist/ML developer,Russia,None,None
2,Junior Data scientist/Machine learning,Аналитик,СБЕР,Москва,NaN,NaN,Обучение классических моделей машинного обучен...,Анализ и использование различных источников да...,От 1 года до 3 лет,False,...,81456638,NaN,[],fullDay,<p>Ищем Data scientist в Блок Сервисы. Это Бло...,Junior Data scientist/Machine learning Аналитик,Data Scientist/ML developer,Russia,None,None
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,2968.0,3561.0,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,True,...,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",fullDay,<p><strong>Задачи:</strong></p> <ul> <li>Приду...,Machine Learning Engineer (ML) Другое,Data Scientist/ML developer,Russia,None,None
5,Machine Learning Engineer,"Программист, разработчик",НеоБИТ,Санкт-Петербург,NaN,NaN,"Знания Мат. аппарата (мат. статистики, теории ...",Исследование и реализация методов предобработк...,От 1 года до 3 лет,False,...,80948681,NaN,"[{'name': 'Machine Learning'}, {'name': 'Compu...",fullDay,"<p>Приглашаем в команду специалиста по ML,кото...","Machine Learning Engineer Программист, разрабо...",Data Scientist/ML developer,Russia,None,None
6,Data scientist / Machine learning engineer (ко...,"Программист, разработчик",VK,Санкт-Петербург,NaN,NaN,Математически формулировать бизнес-задачи. С п...,Использовать огромное количество разных данных...,От 1 года до 3 лет,False,...,81833434,NaN,"[{'name': 'Python'}, {'name': 'Machine Learnin...",flexible,<p>Мы применяем алгоритмы машинного обучения к...,Data scientist / Machine learning engineer (ко...,Data Scientist/ML developer,Russia,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,Middle Дата Аналитик в продукт Личные Финансы ...,Аналитик,МТС,Москва,NaN,NaN,Знание статистических методов и умение проводи...,"Проводить A/B тесты (дизайн, анализ, интерпрет...",От 3 до 6 лет,False,...,81498254,NaN,"[{'name': 'A/B тесты'}, {'name': 'SQL'}, {'nam...",fullDay,"<p><strong>Big Data МТС</strong> – место, где ...",Middle Дата Аналитик в продукт Личные Финансы ...,Data analyst,Russia,None,None
2140,Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,Опыт написания технических заданий на разработ...,Участие в создании бизнес-метрик продукта. Пло...,От 3 до 6 лет,False,...,81088795,NaN,[],remote,"<p><strong>Итак, тебе предстоит:</strong></p> ...","Аналитик больших данных BI-аналитик, аналитик ...",Data analyst,Russia,None,None
2141,Старший Аналитик больших данных,"BI-аналитик, аналитик данных",билайн,Москва,NaN,NaN,Опыт работы аналитиком от 2 лет. Развитые комм...,"Сбор, анализ и разработка (уточнение и детализ...",От 1 года до 3 лет,False,...,78357091,NaN,"[{'name': 'Python'}, {'name': 'Spark'}, {'name...",fullDay,"<p>Создание и развитие цифровых инструментов, ...","Старший Аналитик больших данных BI-аналитик, а...",Data analyst,Russia,None,None
2142,Эксперт по исследованиям больших данных,"BI-аналитик, аналитик данных",UNITEL LLC,Ташкент,NaN,NaN,Опыт 3+ года. Master's or Ph.D в области инфор...,"Оценка целей бизнес задач, бизнес метрик, алго...",От 3 до 6 лет,False,...,81459076,NaN,[],fullDay,<p><strong>Обязанности:</strong></p> <ul> <li>...,Эксперт по исследованиям больших данных BI-ана...,Data analyst,Uzbekistan,None,None


In [82]:
df.to_csv('reached_data2.csv', index=False)